In [1]:
import sys
sys.path.append('../..')

from importlib import reload
import numpy as np
from matplotlib import pyplot as plt
import eight_qubit_design as eqd
import QCreator.elements as elements
import QCreator.auxiliary_functions as auxfuncs

In [2]:
from importlib import reload
reload(eqd)

<module 'eight_qubit_design' from '/home/alena/QCreator/designs/8qubit_desigh/eight_qubit_design.py'>

# Create design

## Chip edge ground  and contact pads

In [3]:
# Ilya wants:
# 1. increase length of coupler between purcell and TL up to 650 mu m 
# 2. Bridges after pads

In [4]:
# eqd.sample.draw_design()

In [5]:
# eqd.sample.watch()

## Add coaxmons

In [6]:
eqd.sample.add(eqd.coaxmon1)
eqd.sample.add(eqd.coaxmon2)
eqd.sample.add(eqd.coaxmon3)
eqd.sample.add(eqd.coaxmon4)

eqd.sample.add(eqd.coaxmon5)
eqd.sample.add(eqd.coaxmon6)
eqd.sample.add(eqd.coaxmon7)
eqd.sample.add(eqd.coaxmon8)

eqd.sample.draw_design()
# eqd.sample.watch()

In [7]:
# eqd.sample.watch()

In [8]:
# define airbridges parameters
airbridge = elements.AirBridgeGeometry(pad_width=36, pad_length=22, pad_distance=62,
                                       narrow_width=20, narrow_length=46, sm_pad_length=10,
                                       sm_pad_distance=70, layer_configuration=eqd.sample.layer_configuration)
# resonator_min_bridge_spacing = 100
resonator_min_bridge_spacing = 50
min_bridge_spacing = 300

In [9]:
# eqd.sample.write_to_gds()

## SQUID tunable couplers 

In [10]:
from QCreator.elements.squid_in_line import SquidInLine, rotate_point
from QCreator.elements.JJ_in_line import JJInLine
from QCreator.elements.gflux_squid import GFluxControllableSquid

In [11]:
jc = 1e-6
length = 60

coupler_s = 26
flux_w = 10
flux_s = 6
flux_g = 20

side_squid_offset = 200
qubit_position_offset = 900
tunable_coupler_length =1600

side_squid_offset = 200

squid_coupler_horizontal =  {'jjs_height': 0.3,
                             'jjs_width': 0.3,
                             'jj_lead_width':0.5,
                             'chain_width':0.5,
                             'ics': 0.3*0.3*jc,
                             'icb': 0.5*0.5*jc,
                             'chain_junctions': 1,
                             'lm': 3.3e-12,
                             'squid_orientation': 'horizontal',
                             'jjs_distance': 0.16,
                             'chain_jj_distance': 0.16}
squid_coupler_vertical =    {'jjs_height': 0.3,
                             'jjs_width': 0.3,
                             'jj_lead_width':0.5,
                             'chain_width':0.5,
                             'ics': 0.3*0.3*jc,
                             'icb': 0.5*0.5*jc,
                             'chain_junctions': 1,
                             'lm': 3.3e-12,
                             'squid_orientation': 'vertical',
                             'jjs_distance': 0.16,
                             'chain_jj_distance': 0.16}

In [12]:
nu_bottom_squid_lines = 3
bottom_squid_lines_positions = []
for i in range(nu_bottom_squid_lines):
    position = (eqd.sample.chip_geometry.sample_horizontal_size/2 + (i-1) * tunable_coupler_length,
                eqd.sample.chip_geometry.sample_vertical_size/2-qubit_position_offset)
    bottom_squid_lines_positions.append(position)

nu_top_squid_lines = 3
top_squid_lines_positions = []
for i in range(nu_top_squid_lines):
    position = (eqd.sample.chip_geometry.sample_horizontal_size/2 + (i-1) * tunable_coupler_length,
                eqd.sample.chip_geometry.sample_vertical_size/2+qubit_position_offset)
    top_squid_lines_positions.append(position)

nu_vertical_squid_lines = 4
vertical_squid_lines_positions = []
for i in range(nu_vertical_squid_lines):
    if i % 2 == 0:
        position = (eqd.sample.chip_geometry.sample_horizontal_size/2 + (i-3/2) * tunable_coupler_length,
                    eqd.sample.chip_geometry.sample_vertical_size/2-side_squid_offset)
    else:
        position = (eqd.sample.chip_geometry.sample_horizontal_size/2 + (i-3/2) * tunable_coupler_length,
                    eqd.sample.chip_geometry.sample_vertical_size/2+side_squid_offset)

    vertical_squid_lines_positions.append(position)

In [13]:
bottom_squid_lines = []
for squid_line_id, sl_position in zip(range(nu_bottom_squid_lines), bottom_squid_lines_positions):
    squid_line = GFluxControllableSquid(name='squid_inline_bottom' + str(squid_line_id), position=sl_position, w=8, s=coupler_s, g=30,
                                        layer_configuration=eqd.sample.layer_configuration, squid_params=squid_coupler_horizontal,
                                        flux_w=flux_w, flux_s=flux_s, flux_g=flux_g)
    bottom_squid_lines.append(squid_line)


top_squid_lines = []
for squid_line_id, sl_position in zip(range(nu_top_squid_lines), top_squid_lines_positions):
    squid_line = GFluxControllableSquid(name='squid_inline_top' + str(squid_line_id), position=sl_position, w=8, s=coupler_s, g=30, invert_y=True,
                                        layer_configuration=eqd.sample.layer_configuration, squid_params=squid_coupler_horizontal,
                                        flux_w=flux_w, flux_s=flux_s, flux_g=flux_g)
    top_squid_lines.append(squid_line)

vertical_squid_lines = []
for squid_line_id, sl_position in zip(range(nu_vertical_squid_lines), vertical_squid_lines_positions):
    if squid_line_id % 2 == 0:
        invert_x_=True
        
    else:
        invert_x_=False
    squid_line = GFluxControllableSquid(name='squid_inline_vertical' + str(squid_line_id), position=sl_position, w=8, s=coupler_s, g=30, invert_x=invert_x_,
                                        layer_configuration=eqd.sample.layer_configuration, squid_params=squid_coupler_vertical,
                                        flux_w=flux_w, flux_s=flux_s, flux_g=flux_g)
    vertical_squid_lines.append(squid_line)


{'jjs_height': 0.3, 'jjs_width': 0.3, 'jj_lead_width': 0.5, 'chain_width': 0.5, 'ics': 9e-08, 'icb': 2.5e-07, 'chain_junctions': 1, 'lm': 3.3e-12, 'squid_orientation': 'horizontal', 'jjs_distance': 0.16, 'chain_jj_distance': 0.16}
{'jjs_height': 0.3, 'jjs_width': 0.3, 'jj_lead_width': 0.5, 'chain_width': 0.5, 'ics': 9e-08, 'icb': 2.5e-07, 'chain_junctions': 1, 'lm': 3.3e-12, 'squid_orientation': 'horizontal', 'jjs_distance': 0.16, 'chain_jj_distance': 0.16}
{'jjs_height': 0.3, 'jjs_width': 0.3, 'jj_lead_width': 0.5, 'chain_width': 0.5, 'ics': 9e-08, 'icb': 2.5e-07, 'chain_junctions': 1, 'lm': 3.3e-12, 'squid_orientation': 'horizontal', 'jjs_distance': 0.16, 'chain_jj_distance': 0.16}
{'jjs_height': 0.3, 'jjs_width': 0.3, 'jj_lead_width': 0.5, 'chain_width': 0.5, 'ics': 9e-08, 'icb': 2.5e-07, 'chain_junctions': 1, 'lm': 3.3e-12, 'squid_orientation': 'horizontal', 'jjs_distance': 0.16, 'chain_jj_distance': 0.16}
{'jjs_height': 0.3, 'jjs_width': 0.3, 'jj_lead_width': 0.5, 'chain_width': 0

In [14]:
for squid_line in bottom_squid_lines + top_squid_lines + vertical_squid_lines:
    eqd.sample.add(squid_line)
    
eqd.sample.draw_design()
# eqd.sample.watch()

In [15]:
# Connections between qubits and couplers
coupler_bridge_spacing = 500
bottom_connections = zip((eqd.coaxmon1, eqd.coaxmon2, eqd.coaxmon5, eqd.coaxmon6),
                      (bottom_squid_lines[0], bottom_squid_lines[0],
                       bottom_squid_lines[2], bottom_squid_lines[2]),
                      ('coupler3', 'coupler3', 'coupler4', 'coupler3'),
                      ('port1', 'port2', 'port1', 'port2'))


for qubit, squid_line, q_port, sl_port in bottom_connections:
    eqd.sample.connect_cpw(qubit, squid_line, q_port, sl_port, 'jj-squid cpw', [], airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing)
    
top_connections = zip((eqd.coaxmon4, eqd.coaxmon3, eqd.coaxmon8, eqd.coaxmon7),
                      (top_squid_lines[0], top_squid_lines[0],
                       top_squid_lines[2], top_squid_lines[2]),
                      ('coupler3', 'coupler4', 'coupler3', 'coupler3'),
                      ('port1', 'port2', 'port1', 'port2'))

for qubit, squid_line, q_port, sl_port in top_connections:
    eqd.sample.connect_cpw(qubit, squid_line, q_port, sl_port, 'jj-squid cpw', [], airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing)


In [16]:
eqd.sample.draw_design()
# eqd.sample.watch()

In [17]:
# eqd.sample.watch()

## Cross line bridges

In [18]:
from QCreator.elements import cross_lines

In [19]:
nu_reflection_cross_lines = 4
reflection_cross_lines_positions = []
for i in range(nu_reflection_cross_lines):
    position = (eqd.sample.chip_geometry.sample_horizontal_size/2 + (i-3/2) *
                tunable_coupler_length, eqd.sample.chip_geometry.sample_vertical_size/2)
    reflection_cross_lines_positions.append(position)

nu_flux_cross_lines = 2
flux_cross_line_offset = 450
flux_cross_lines_positions = [(eqd.sample.chip_geometry.sample_horizontal_size/2 - flux_cross_line_offset,
                               eqd.sample.chip_geometry.sample_vertical_size/2+qubit_position_offset),
                              (eqd.sample.chip_geometry.sample_horizontal_size/2 + flux_cross_line_offset,
                              eqd.sample.chip_geometry.sample_vertical_size/2-qubit_position_offset)]

In [20]:
cross_bridges = []
for cross_line_id, cross_line_position in zip(range(nu_reflection_cross_lines), reflection_cross_lines_positions):
    cross_line = cross_lines.CrossLinesViaAirbridges('reflection_cross_line'+str(cross_line_id),
                                                     cross_line_position, 0, eqd.tl_core, eqd.tl_gap, eqd.tl_ground,
                                                     8, coupler_s, 30, 46, airbridge)
    eqd.sample.add(cross_line)
    cross_bridges.append(cross_line)

for cross_line_id, cross_line_position in zip(range(nu_flux_cross_lines), flux_cross_lines_positions):
    cross_line = cross_lines.CrossLinesViaAirbridges('flux_cross_line'+str(cross_line_id),
                                                     cross_line_position, np.pi/2, eqd.tl_core, eqd.tl_gap, eqd.tl_ground,
                                                     8, coupler_s, 30, 46, airbridge)
    eqd.sample.add(cross_line)
    cross_bridges.append(cross_line)

In [21]:
eqd.sample.draw_design()
# eqd.sample.watch()

In [22]:
# Connection between cross lines and couplers and qubits
cross_bridges_connections1 = zip((eqd.coaxmon1, eqd.coaxmon4, cross_bridges[0], eqd.coaxmon5, eqd.coaxmon8, cross_bridges[2]),
                                 (vertical_squid_lines[0], cross_bridges[0],
                                  vertical_squid_lines[0], vertical_squid_lines[2], cross_bridges[2],
                                  vertical_squid_lines[2]),
                                 ('coupler2', 'coupler2', 'bottom_2',
                                  'coupler2', 'coupler2', 'bottom_2'),
                                 ('port1', 'bottom_1', 'port2', 'port1', 'bottom_1', 'port2'))


for elem1, elem2, port1, port2 in cross_bridges_connections1:
    eqd.sample.connect_cpw(elem1, elem2, port1, port2, 'jj-squid cpw', [], airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing)
    
cross_bridges_connections2 = zip((eqd.coaxmon7, eqd.coaxmon6, cross_bridges[3], eqd.coaxmon3, eqd.coaxmon2, cross_bridges[1]),
                                 (vertical_squid_lines[3], cross_bridges[3],
                                  vertical_squid_lines[3], vertical_squid_lines[1], cross_bridges[1],
                                  vertical_squid_lines[1]),
                                 ('coupler2', 'coupler2', 'bottom_1', 'coupler2', 'coupler2', 'bottom_1'),
                                 ('port2', 'bottom_2', 'port1', 'port2', 'bottom_2', 'port1'))


for elem1, elem2, port1, port2 in cross_bridges_connections2:
    eqd.sample.connect_cpw(elem1, elem2, port1, port2, 'jj-squid cpw', [], airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing)

cross_bridges_connections3 = zip((eqd.coaxmon2, eqd.coaxmon5, cross_bridges[5]),
                                 (bottom_squid_lines[1], cross_bridges[5],
                                  bottom_squid_lines[1]),
                                 ('coupler4', 'coupler3', 'bottom_1'),
                                 ('port1', 'bottom_2', 'port2'))


for elem1, elem2, port1, port2 in cross_bridges_connections3:
    eqd.sample.connect_cpw(elem1, elem2, port1, port2, 'jj-squid cpw', [], airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing)
    
cross_bridges_connections4 = zip((eqd.coaxmon8, eqd.coaxmon3, cross_bridges[4]),
                                 (top_squid_lines[1], cross_bridges[4],
                                  top_squid_lines[1]),
                                 ('coupler4', 'coupler3', 'bottom_2'),
                                 ('port2', 'bottom_1', 'port1'))


for elem1, elem2, port1, port2 in cross_bridges_connections4:
    eqd.sample.connect_cpw(elem1, elem2, port1, port2, 'jj-squid cpw', [], airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing)

In [23]:
eqd.sample.draw_design()
# eqd.sample.watch()

In [24]:
# eqd.sample.watch()

## Add extra airbridges near pads and cross lines

In [25]:
TL_bridges_ports = []
TL_extra_bridges = []
# bridge near pads
for elem in [eqd.pads_left[1], eqd.pads_right[1]]:
    bridge, port = eqd.sample.airbridge(
    elem, 'port', name='Airbridge near pad %s'% elem.name, geometry=airbridge)
    TL_extra_bridges.append(bridge)
    TL_bridges_ports.append(port)
    
# bridge near cross lines
for elem_id, elem in enumerate(cross_bridges):    
    bridge, port = eqd.sample.airbridge(
    elem, 'top_1', name='Airbridge near cross line %s'% elem.name, geometry=airbridge)
    TL_extra_bridges.append(bridge)
    TL_bridges_ports.append(port)
    
    bridge, port = eqd.sample.airbridge(
    elem, 'top_2', name='Airbridge near cross line %s'% elem.name, geometry=airbridge)
    TL_extra_bridges.append(bridge)
    TL_bridges_ports.append(port)
    
eqd.sample.draw_design()
# eqd.sample.watch()

In [26]:
# eqd.sample.watch()

## Add flux lines 

In [27]:
# eqd.sample.connect_cpw(eqd.pads_top[2],
#                        cross_bridges[4],
#                        'port',
#                        'top_2',
#                        'flux_control_line_qubit_' + cross_bridges[2].name,
#                        points=[],
#                        airbridge=airbridge,
#                        min_spacing=min_bridge_spacing,
#                        r=80)
# eqd.sample.connect_cpw(eqd.pads_bottom[4],
#                        cross_bridges[5],
#                        'port',
#                        'top_1',
#                        'flux_control_line_qubit_' + eqd.pads_bottom[3].name,
#                        points=[],
#                        airbridge=airbridge,
#                        min_spacing=min_bridge_spacing,
#                        r=50)
eqd.sample.connect_cpw(eqd.pads_top[2],
                       TL_extra_bridges[11],
                       'port',
                       TL_bridges_ports[11],
                       'flux_control_line_qubit_' + cross_bridges[2].name,
                       points=[],
                       airbridge=airbridge,
                       min_spacing=min_bridge_spacing,
                       r=80)
eqd.sample.connect_cpw(eqd.pads_bottom[4],
                       TL_extra_bridges[12],
                       'port',
                       TL_bridges_ports[12],
                       'flux_control_line_qubit_' + eqd.pads_bottom[3].name,
                       points=[],
                       airbridge=airbridge,
                       min_spacing=min_bridge_spacing,
                       r=50)

[CPW "flux_control_line_qubit_pad-bottom-3s0", l=277.495,
 CPW "flux_control_line_qubit_pad-bottom-3s1", l=443.562,
 CPW "flux_control_line_qubit_pad-bottom-3s2", l=277.601]

In [28]:
eqd.sample.draw_design()
# eqd.sample.watch()

In [29]:
flux_pads_qubits_and_couplers = [
    (vertical_squid_lines[3],  eqd.pads_right[0], []),
#     (,  eqd.pads_right[1], []), to readout
    (eqd.coaxmon6,  eqd.pads_right[2], []),
    
    (eqd.coaxmon4,  eqd.pads_left[0], []),
#     (,  eqd.pads_left[1], []), to readout
    (vertical_squid_lines[0],  eqd.pads_left[2], []),
    
    (eqd.coaxmon1,  eqd.pads_bottom[0], []),
    (bottom_squid_lines[0],  eqd.pads_bottom[1], []),
    (eqd.coaxmon2,  eqd.pads_bottom[2], []),
    (bottom_squid_lines[1],  eqd.pads_bottom[3], []),
#     (,  eqd.pads_bottom[4], []), bridge
    (eqd.coaxmon5,  eqd.pads_bottom[5], []),
    (bottom_squid_lines[2],  eqd.pads_bottom[6], []),
    
    (top_squid_lines[0],  eqd.pads_top[0], []),
    (eqd.coaxmon3,  eqd.pads_top[1], []),
#     (,  eqd.pads_top[2], []), bridge
    (top_squid_lines[1],  eqd.pads_top[3], []),
    (eqd.coaxmon8,  eqd.pads_top[4], []),
    (top_squid_lines[2],  eqd.pads_top[5], []),
    (eqd.coaxmon7,  eqd.pads_top[6], [])
    ]

In [30]:
for elem, pad, points in flux_pads_qubits_and_couplers:  
    bridge, port = eqd.sample.airbridge(
            elem, 'flux', name='Airbridge over %s qubit flux coupler' % elem.name, geometry=airbridge)
    
    narrowing_length = 25
    flux_line_narrowing_position = eqd.sample.cpw_shift(bridge, port, narrowing_length/2)[0]

    flux_line_narrowing = eqd.elements.Narrowing(name='flux_line_narrowing', position=flux_line_narrowing_position, 
                                             orientation=bridge.terminals[port].orientation+np.pi, 
                                             w1=bridge.terminals[port].w, 
                                             s1=bridge.terminals[port].s, g1=bridge.terminals[port].g,
                                             w2=eqd.tl_core, s2=eqd.tl_gap, g2=eqd.tl_ground, 
                                             layer_configuration=eqd.sample.layer_configuration, length=narrowing_length)
    eqd.sample.add(flux_line_narrowing)
    eqd.sample.connect(flux_line_narrowing, 'port1', bridge, port)
    eqd.sample.connect_cpw(pad, flux_line_narrowing, 'port', 'port2', 'flux_control_line_qubit_'+elem.name, points=points,
                          airbridge=airbridge, min_spacing=min_bridge_spacing, r=80)
    print(elem)

In [31]:
coplers_and_cross_bridges = [(vertical_squid_lines[1],  cross_bridges[4], []),
                             (vertical_squid_lines[2],  cross_bridges[5], [])
                             ]

In [32]:
flux_line_narrowings = []
for elem, cross_bridge,points in coplers_and_cross_bridges:  
    bridge, port = eqd.sample.airbridge(
            elem, 'flux', name='Airbridge over %s qubit flux coupler' % elem.name, geometry=airbridge)
    narrowing_length = 25
    flux_line_narrowing_position = eqd.sample.cpw_shift(bridge, port, narrowing_length/2)[0]
    flux_line_narrowing = eqd.elements.Narrowing(name='flux_line_narrowing', position=flux_line_narrowing_position, 
                                             orientation=bridge.terminals[port].orientation+np.pi, 
                                             w1=bridge.terminals[port].w, 
                                             s1=bridge.terminals[port].s, g1=bridge.terminals[port].g,
                                             w2=eqd.tl_core, s2=eqd.tl_gap, g2=eqd.tl_ground, 
                                             layer_configuration=eqd.sample.layer_configuration, length=narrowing_length)
    eqd.sample.add(flux_line_narrowing)
    flux_line_narrowings.append(flux_line_narrowing)
    eqd.sample.connect(flux_line_narrowing, 'port1', bridge, port)

In [33]:
eqd.sample.connect_cpw(TL_extra_bridges[10],
                       flux_line_narrowings[0],
                       TL_bridges_ports[10],
                       'port2',
                       'flux_control_line_qubit_' + elem.name,
                       points=[(eqd.coaxmon3.center[0] + 130,
                                eqd.coaxmon3.center[1] - 420)],
                       airbridge=airbridge,
                       min_spacing=min_bridge_spacing/2,
                       r=80)
eqd.sample.connect_cpw(TL_extra_bridges[13],
                       flux_line_narrowings[1],
                       TL_bridges_ports[13],
                       'port2',
                       'flux_control_line_qubit_' + elem.name,
                       points=[(eqd.coaxmon5.center[0] - 130,
                                eqd.coaxmon5.center[1] + 420)],
                       airbridge=airbridge,
                       min_spacing=min_bridge_spacing/2,
                       r=80)

[CPW "flux_control_line_qubit_squid_inline_vertical2s0", l=175.347,
 CPW "flux_control_line_qubit_squid_inline_vertical2s1", l=478.949]

In [34]:
eqd.sample.draw_design()
# eqd.sample.watch()

In [35]:
# eqd.sample.watch()

## Add TL with couplers between line and purcells 

In [36]:
tunable_coupler_length =1600

In [37]:
# all couplers between TL and purcells 
TL_purcell_couplers = []

In [38]:
delta_from_fanout = eqd.resonator_ground + 2 * eqd.resonator_gap + eqd.resonator_core

In [39]:
nu_couplers = 2
tl_coupler_length1 = 600  # long coupling
tl_coupler_length2 = 300 - 2 * delta_from_fanout # short double coupling
tl_coupler_delta_length = tl_coupler_length1 - tl_coupler_length2
coupler_width = eqd.tl_ground + eqd.tl_gap + eqd.tl_core + \
    eqd.resonator_gap + eqd.resonator_core + eqd.resonator_gap + eqd.resonator_ground

In [40]:
# coupler_start_x_offset = 300
coupler_start_x_offset = 450
coupler_start_y_offset = coupler_width / 2 - (eqd.tl_ground + eqd.tl_gap + eqd.tl_core/ 2)
coupler_start_x1 = eqd.pads_left[1].terminal.position[0] + coupler_start_x_offset
coupler_start_y1 = eqd.pads_left[1].terminal.position[1] + coupler_start_y_offset

coupler_start_x2 = eqd.pads_right[1].terminal.position[0] - coupler_start_x_offset
coupler_start_y2 = eqd.pads_right[1].terminal.position[1] - coupler_start_y_offset

In [41]:
coupler_positions = [[(coupler_start_x1, coupler_start_y1),
                      (coupler_start_x1 + tl_coupler_length1, coupler_start_y1)],
                     [(coupler_start_x2, coupler_start_y2),
                      (coupler_start_x2 - tl_coupler_length1, coupler_start_y2)]]

In [42]:
coupler_w = [eqd.tl_core, eqd.resonator_core]
coupler_s = [eqd.tl_gap, eqd.resonator_gap, eqd.resonator_gap]
coupler_ground = eqd.resonator_ground

### Add couplers 0 and 1 

In [43]:
for coupler_id, position in zip(range(nu_couplers), coupler_positions):
    main_coupler = elements.CPWCoupler('TL-purcell coupler',
                                       position,
                                       coupler_w,
                                       coupler_s,
                                       coupler_ground,
                                       eqd.sample.layer_configuration,
                                       r=0)

    fanout1 = eqd.sample.fanout(o=main_coupler,
                                port='port1',
                                name='TL-purcell-fanout1',
                                grouping=[0, 1])
    fanout2 = eqd.sample.fanout(o=main_coupler,
                                port='port2',
                                name='TL-purcell-fanout2',
                                grouping=[0, 1])
    eqd.sample.add(main_coupler)
    TL_purcell_couplers.append([fanout1, fanout2])

In [44]:
eqd.sample.draw_design()
# eqd.sample.watch()

/home/alena/.local/lib/python3.9/site-packages/gdspy/path.py:616: UserWarning: [GDSPY] Possible inconsistency found in `FlexPath` due to sharp corner.
  warnings.warn(


In [45]:
double_coupler_w = [eqd.resonator_core, eqd.tl_core, eqd.resonator_core]
double_coupler_s = [eqd.resonator_gap, eqd.resonator_gap, eqd.resonator_gap, eqd.resonator_gap]

### Add couplers (2, 3) and (4, 5)

In [46]:
inner_coupler_start_y = eqd.pads_left[1].terminal.position[1]
for i in [0, 2]:
    inner_coupler_start_x = eqd.sample.chip_geometry.sample_horizontal_size / \
        2 + (i-1) * tunable_coupler_length

    coupler_top_position = [
        (inner_coupler_start_x + tl_coupler_length2 / 2 + delta_from_fanout,
         inner_coupler_start_y + coupler_start_y_offset),
        (inner_coupler_start_x + tl_coupler_length1 - tl_coupler_length2 / 2,
         inner_coupler_start_y + coupler_start_y_offset)
    ]
    
    print('coupler_top_position', coupler_top_position)

    coupler_bottom_position = [
        (inner_coupler_start_x - tl_coupler_length1 + tl_coupler_length2 / 2,
         inner_coupler_start_y - coupler_start_y_offset),
        (inner_coupler_start_x - tl_coupler_length2 / 2 - delta_from_fanout,
         inner_coupler_start_y - coupler_start_y_offset)
    ]
    
    print('coupler_bottom_position',coupler_bottom_position)

    double_coupler_position = [(inner_coupler_start_x - tl_coupler_length2 / 2,
                                inner_coupler_start_y),
                               (inner_coupler_start_x + tl_coupler_length2 / 2,
                                inner_coupler_start_y)]

    main_coupler_top = elements.CPWCoupler('main_coupler_top',
                                           coupler_top_position,
                                           coupler_w,
                                           coupler_s,
                                           coupler_ground,
                                           eqd.sample.layer_configuration,
                                           r=0)

    main_coupler_bottom = elements.CPWCoupler('main_coupler_bottom',
                                              coupler_bottom_position,
                                              coupler_w,
                                              coupler_s,
                                              coupler_ground,
                                              eqd.sample.layer_configuration,
                                              r=0)

    double_coupler = elements.CPWCoupler('double_coupler',
                                         double_coupler_position,
                                         double_coupler_w,
                                         double_coupler_s,
                                         coupler_ground,
                                         eqd.sample.layer_configuration,
                                         r=0)
    eqd.sample.add(main_coupler_top)
    eqd.sample.add(main_coupler_bottom)
    eqd.sample.add(double_coupler)

    mcb_fanout1 = eqd.sample.fanout(o=main_coupler_bottom,
                                    port='port1',
                                    name='mcb_fanout1',
                                    grouping=[1, 2])
    mct_fanout2 = eqd.sample.fanout(o=main_coupler_top,
                                    port='port2',
                                    name='mct_fanout2',
                                    grouping=[0, 1])

    dc_fanout1 = eqd.sample.fanout(o=double_coupler,
                                   port='port2',
                                   name='dc_fanout1',
                                   grouping=[1, 3])
    dc_fanout2 = eqd.sample.fanout(o=double_coupler,
                                   port='port1',
                                   name='dc_fanout2',
                                   grouping=[0, 2])
    
    TL_purcell_couplers.append([mcb_fanout1, dc_fanout1])
    TL_purcell_couplers.append([mct_fanout2, dc_fanout2])
    
    
    extra_line_offset = (eqd.resonator_core + eqd.resonator_ground) / 2 + eqd.resonator_gap
    
    extra_bottom_line = elements.CPWCoupler('extra_bottom_line',
                                     [(main_coupler_bottom.terminals['port2'].position[0],
                                       inner_coupler_start_y - coupler_start_y_offset),
                                      (main_coupler_bottom.terminals['port2'].position[0] + eqd.resonator_ground,
                                       inner_coupler_start_y - coupler_start_y_offset)],
                                     coupler_w,
                                     coupler_s,
                                     eqd.resonator_ground,
                                     eqd.sample.layer_configuration,
                                     r=0)
    
    
    
    eqd.sample.add(extra_bottom_line)
    
    eqd.sample.connect(o1=main_coupler_bottom, p1='port2', o2=extra_bottom_line, p2='port1')
    eqd.sample.connect(o1=dc_fanout1, p1='center', o2=extra_bottom_line, p2='port2')

    
    
    extra_top_line = elements.CPWCoupler('extra_top_line',
                                     [(main_coupler_top.terminals['port1'].position[0]- eqd.resonator_ground,
                                       inner_coupler_start_y + coupler_start_y_offset),
                                      (main_coupler_top.terminals['port1'].position[0] ,
                                       inner_coupler_start_y + coupler_start_y_offset)],
                                     coupler_w,
                                     coupler_s,
                                     eqd.resonator_ground,
                                     eqd.sample.layer_configuration,
                                     r=0)
    
    eqd.sample.add(extra_top_line)
    
    eqd.sample.connect(o1=main_coupler_top, p1='port1', o2=extra_top_line, p2='port2')
    eqd.sample.connect(o1=extra_top_line, p1='port1', o2=dc_fanout2, p2='center')


coupler_top_position [(3250.0, 2358.0), (3582.0, 2358.0)]
coupler_bottom_position [(2618.0, 2342.0), (2950.0, 2342.0)]
coupler_top_position [(6450.0, 2358.0), (6782.0, 2358.0)]
coupler_bottom_position [(5818.0, 2342.0), (6150.0, 2342.0)]


In [47]:
eqd.sample.draw_design()
# eqd.sample.watch()

In [48]:
# sys, connections, elements_ = eqd.sample.get_tls(cutoff=1e11)

In [49]:
# eqd.sample.get_topology()

In [50]:
# coupler_start_x_offset_ = 250
coupler_start_x_offset_ = 280
inner_coupler_start_y = eqd.pads_left[1].terminal.position[1]

central_couplers_positions = [
    [(eqd.sample.chip_geometry.sample_horizontal_size / 2 -
      (tunable_coupler_length / 2 - coupler_start_x_offset_),
      inner_coupler_start_y - 40),
     (eqd.sample.chip_geometry.sample_horizontal_size / 2 -
      (tunable_coupler_length / 2 - coupler_start_x_offset_) +
      tl_coupler_length1, inner_coupler_start_y - 40)],
    [(eqd.sample.chip_geometry.sample_horizontal_size / 2 +
      (tunable_coupler_length / 2 - coupler_start_x_offset_),
      inner_coupler_start_y + 40),
     (eqd.sample.chip_geometry.sample_horizontal_size / 2 +
      (tunable_coupler_length / 2 - coupler_start_x_offset_) -
      tl_coupler_length1, inner_coupler_start_y + 40)]
]

### Add coupler 6 and 7 

In [51]:
for i, position in enumerate(central_couplers_positions):
    main_coupler = elements.CPWCoupler('TL-purcell coupler',
                                       position,
                                       coupler_w,
                                       coupler_s,
                                       coupler_ground,
                                       eqd.sample.layer_configuration,
                                       r=0)
    
    eqd.sample.add(main_coupler)
    
    fanout1 = eqd.sample.fanout(o=main_coupler,
                                    port='port1',
                                    name='TL-purcell-fanout1',
                                    grouping=[1, 2])
    fanout2 = eqd.sample.fanout(o=main_coupler,
                                    port='port2',
                                    name='TL-purcell-fanout2',
                                    grouping=[1, 2])
    
    TL_purcell_couplers.append([fanout1, fanout2])

### Connect all parts of the left reflection TL 

In [52]:
elem1 = (TL_purcell_couplers[0][0], TL_purcell_couplers[0][1],
         TL_purcell_couplers[2][0], TL_purcell_couplers[3][0],
         TL_purcell_couplers[6][0])
elem2 = (TL_extra_bridges[0], TL_extra_bridges[2], TL_extra_bridges[3],
         TL_extra_bridges[4], TL_extra_bridges[5])
port1 = ('center', 'center', 'center', 'center', 'center')
port2 = ('port1', 'port1', 'port1', 'port1', 'port1')

left_TL_connections = zip(elem1, elem2, port1, port2)

In [53]:
for coupler, bridge, port1, port2 in left_TL_connections:
    eqd.sample.connect_cpw(coupler,
                           bridge,
                           port1,
                           port2,
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=80)

In [54]:
# add open end
eqd.sample.open_end(TL_purcell_couplers[6][1], 'center', 'left reflection line end')

OpenEnd left reflection line end

### Connect all parts of the right reflection TL 

In [55]:
elem1 = (TL_purcell_couplers[1][0], TL_purcell_couplers[1][1],
         TL_purcell_couplers[5][0], TL_purcell_couplers[4][0],
         TL_purcell_couplers[7][0])
elem2 = (TL_extra_bridges[1], TL_extra_bridges[9], TL_extra_bridges[8],
         TL_extra_bridges[7], TL_extra_bridges[6])
port1 = ('center', 'center', 'center', 'center', 'center')
port2 = ('port1', 'port1', 'port1', 'port1', 'port1')

right_TL_connections = zip(elem1, elem2, port1, port2)

In [56]:
for coupler, bridge, port1, port2 in right_TL_connections:
    eqd.sample.connect_cpw(coupler,
                           bridge,
                           port1,
                           port2,
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=80)

In [57]:
# add open end
eqd.sample.open_end(TL_purcell_couplers[7][1], 'center', 'right reflection line end')

OpenEnd right reflection line end

In [58]:
eqd.sample.draw_design()
# eqd.sample.watch()

In [59]:
# eqd.sample.watch()

In [60]:
# eqd.sample.write_to_gds()

In [122]:
# Расстояние между двумя cross bridge 1330
# На один каплер с учетом fanout уходит 664
# На открытый конец считывающей линии уходит 30-40 
# Итого: 664*2 + 2 * 30 = 1388
# Вердикт: не влезает

# Add  purcells and resonators

In [61]:
from QCreator.elements import meander
res_pur_coupler_grounding = 10
purcell_resonator_coupler_length = 200
pur_res_coupler_w = [eqd.resonator_core, 2 * eqd.resonator_ground, eqd.resonator_core]
pur_res_coupler_s = [eqd.resonator_gap for i in range(4)]

### Purcell and resonator for qubit1 

In [62]:
# Add coupler purcell + resonator with fanout and grounding
pur_res_coupler1_position = [(eqd.coaxmon2.center[0] - 470, eqd.coaxmon2.center[1]+445),
                             (eqd.coaxmon2.center[0] - 470 + purcell_resonator_coupler_length, eqd.coaxmon2.center[1]+445)]
pur_res_coupler1 = elements.CPWCoupler('purcell_resonator_coupler1',
                                         pur_res_coupler1_position,
                                         pur_res_coupler_w,
                                         pur_res_coupler_s,
                                         eqd.resonator_ground,
                                         eqd.sample.layer_configuration,
                                         r=0)
eqd.sample.add(pur_res_coupler1)

pur_res_coupler1_fanout = eqd.sample.fanout(o=pur_res_coupler1,
                                            port='port2',
                                            name='res-purcell-coupler-fanout',
                                            grouping=[1, 3])
pur_res_coupler1_ground = eqd.sample.ground(
    o=pur_res_coupler1,
    port='port1',
    name='res-purcell-coupler-grounding',
    grounding_width=res_pur_coupler_grounding,
    grounding_between=[(0, 4)])

pur_res_coupler1_fanout_ground = eqd.sample.ground(
    o=pur_res_coupler1_fanout,
    port='center',
    name='res-purcell-coupler-fanout-grounding',
    grounding_width=res_pur_coupler_grounding,
    grounding_between=[(0, 1)])

In [63]:
# Add resonator for qubit 1
name = 'resonator1'
begin_point = (pur_res_coupler1_fanout.terminals['down'].position[0],
               pur_res_coupler1_fanout.terminals['down'].position[1] - 10)
begin_orientation = - np.pi / 2
end_point = (eqd.coaxmon1.terminals['coupler1'].position[0]+100,
             eqd.coaxmon1.terminals['coupler1'].position[1]+150)
end_orientation =   np.pi / 2
meander_length = 3100
left_limit = 330
right_limit = 0
radius_limit = (40,80)
w = eqd.resonator_core
s = eqd.resonator_gap
g = eqd.resonator_ground
result1, points1, real_points1, resonator1 = meander.meander_creation2(name, begin_point, begin_orientation,
                                    end_point, end_orientation, meander_length,
                                    left_limit, right_limit, radius_limit, w,
                                    s, g, eqd.sample.layer_configuration)

eqd.sample.add(resonator1)

eqd.sample.generate_bridge_over_cpw(name='resonator1_airbridges', o=resonator1, geometry=airbridge,
                                 min_spacing=resonator_min_bridge_spacing, bridge_part_decimation=2)

connection = eqd.sample.connect_cpw(resonator1,
                           pur_res_coupler1_fanout,
                           'port1',
                           'down',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=80)
print(connection[0].length)
print(resonator1.length)

A_max, B_max 227.64530749512187 -38.0982636399815
A_max, B_max 250.4839806213414 -15.259590513761964
A_max, B_max 264.18718449707313 -1.5563866380302471
A_max, B_max 273.32265374756093 7.579082612457569
173.7354443706821
A, K, B =  168.12973517435918 109.47901550578547 5.605709196322919
points [(3641.0, 1843.0), (3641.0, 1629.1929185732017), (3549.645307495122, 1629.1929185732017), (3549.645307495122, 1830.0266265838654), (3458.2906149902437, 1830.0266265838654), (3458.2906149902437, 1629.1929185732017), (3366.9359224853656, 1629.1929185732017), (3366.9359224853656, 1830.0266265838654), (3275.5812299804875, 1830.0266265838654), (3275.5812299804875, 1629.1929185732017), (3184.2265374756093, 1629.1929185732017), (3184.2265374756093, 1830.0266265838654), (3092.871844970731, 1830.0266265838654), (3092.871844970731, 1629.1929185732017), (3001.517152465853, 1629.1929185732017), (3001.517152465853, 1830.0266265838654), (2910.162459960975, 1830.0266265838654), (2910.162459960975, 1629.19291857

In [64]:
connection = eqd.sample.connect_cpw(eqd.coaxmon1,
                           resonator1,
                           'coupler1',
                           'port2',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=80)
print(connection[0].length)

185.21856986336934


In [65]:
# Add purcell closed end 1
name = 'purcell_closed_end1'
begin_point = (TL_purcell_couplers[2][1].terminals['down'].position[0] + 100,
               TL_purcell_couplers[2][1].terminals['down'].position[1] - 100)
begin_orientation = 0
end_point = (pur_res_coupler1_fanout_ground.terminals['narrow'].position[0],
             pur_res_coupler1_fanout_ground.terminals['narrow'].position[1] + 80)
end_orientation =   np.pi
meander_length = 2000
left_limit = 350
right_limit = 480
radius_limit = (40,90)
w = eqd.resonator_core
s = eqd.resonator_gap
g = eqd.resonator_ground

result1, points1, real_points1, purcell_closed_end_1 = meander.meander_creation2(name, begin_point, begin_orientation,
                                    end_point, end_orientation, meander_length,
                                    left_limit, right_limit, radius_limit, w,
                                    s, g, eqd.sample.layer_configuration)
eqd.sample.add(purcell_closed_end_1)
eqd.sample.generate_bridge_over_cpw(name='purcell_closed_end_1_airbridges', o=purcell_closed_end_1, geometry=airbridge,
                                 min_spacing=resonator_min_bridge_spacing, bridge_part_decimation=2)


A_max, B_max 414.75 617.75
996.0685985427538
A, K, B =  399.8033805820643 663.0685985427538 596.2652179606896
points [(3329.0, 2213.0), (3783.053380582064, 2213.0), (3783.053380582064, 2104.5), (3011.4847820393106, 2104.5), (3011.4847820393106, 1996.0), (3662.0, 1996.0)] real points [(3329.0, 2213.0), (3728.803380582064, 2213.0), (3728.803380582064, 2104.5), (3065.7347820393106, 2104.5), (3065.7347820393106, 1996.0), (3662.0, 1996.0)] 0 3.141592653589793


[CPW "purcell_closed_end_1_airbridgess0", l=31.975,
 CPW "purcell_closed_end_1_airbridgess1", l=36.000,
 CPW "purcell_closed_end_1_airbridgess1", l=63.951,
 CPW "purcell_closed_end_1_airbridgess3", l=63.951,
 CPW "purcell_closed_end_1_airbridgess4", l=36.000,
 CPW "purcell_closed_end_1_airbridgess4", l=63.951,
 CPW "purcell_closed_end_1_airbridgess6", l=231.769,
 CPW "purcell_closed_end_1_airbridgess7", l=36.000,
 CPW "purcell_closed_end_1_airbridgess7", l=58.724,
 CPW "purcell_closed_end_1_airbridgess9", l=58.724,
 CPW "purcell_closed_end_1_airbridgess10", l=36.000,
 CPW "purcell_closed_end_1_airbridgess10", l=58.724,
 CPW "purcell_closed_end_1_airbridgess12", l=58.724,
 CPW "purcell_closed_end_1_airbridgess13", l=36.000,
 CPW "purcell_closed_end_1_airbridgess13", l=58.724,
 CPW "purcell_closed_end_1_airbridgess15", l=58.724,
 CPW "purcell_closed_end_1_airbridgess16", l=36.000,
 CPW "purcell_closed_end_1_airbridgess16", l=231.482,
 CPW "purcell_closed_end_1_airbridgess18", l=63.378,
 

In [66]:
# Add purcell open end
name = 'purcell_open_end1'
begin_point = (TL_purcell_couplers[2][0].terminals['up'].position[0],
               TL_purcell_couplers[2][0].terminals['up'].position[1]-10)
begin_orientation = - np.pi / 2
end_point = (2965, 2205)
end_orientation =   np.pi / 2
meander_length = 1400
left_limit = 445
right_limit = 0
radius_limit = (40,90)
w = eqd.resonator_core
s = eqd.resonator_gap
g = eqd.resonator_ground

result1, points1, real_points1, purcell_open_end1 = meander.meander_creation2(name, begin_point, begin_orientation,
                                    end_point, end_orientation, meander_length,
                                    left_limit, right_limit, radius_limit, w,
                                    s, g, eqd.sample.layer_configuration)
eqd.sample.add(purcell_open_end1)
eqd.sample.generate_bridge_over_cpw(name='purcell_open_end1_airbridges', o=purcell_open_end1, geometry=airbridge,
                                 min_spacing=resonator_min_bridge_spacing, bridge_part_decimation=2)


connection = eqd.sample.connect_cpw(purcell_open_end1,
                           TL_purcell_couplers[2][0],
                           'port1',
                           'up',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=80)
print(connection[0].length)

# Add open end for purcell 
eqd.sample.open_end(purcell_open_end1, 'port2', 'purcell1-open-end')

A_max, B_max 344.5 -2.5
A_max, B_max 389.25 42.25
282.91372875185675
A, K, B =  248.9884454732714 184.91372875185675 33.92528327858536
points [(2607.0, 2303.0), (2607.0, 2009.2615545267286), (2696.5, 2009.2615545267286), (2696.5, 2283.675283278585), (2786.0, 2283.675283278585), (2786.0, 2009.2615545267286), (2875.5, 2009.2615545267286), (2875.5, 2283.675283278585), (2965.0, 2283.675283278585), (2965, 2205)] real points [(2607.0, 2303.0), (2607.0, 2054.0115545267286), (2696.5, 2054.0115545267286), (2696.5, 2238.925283278585), (2786.0, 2238.925283278585), (2786.0, 2054.0115545267286), (2875.5, 2054.0115545267286), (2875.5, 2238.925283278585), (2965.0, 2238.9252832785855), (2965, 2205)] -1.5707963267948966 1.5707963267948966
10.0


OpenEnd purcell1-open-end

In [67]:
connection= eqd.sample.connect_cpw(purcell_closed_end_1,
                           pur_res_coupler1_fanout_ground,
                           'port2',
                           'narrow',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=40)
print(connection[0].length)

209.82770192589157


In [68]:
connection =eqd.sample.connect_cpw(TL_purcell_couplers[2][1],
                           purcell_closed_end_1,
                           'down',
                           'port1',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=80)
print(connection[0].length)

164.18075923224458


In [69]:
eqd.sample.draw_design()
# eqd.sample.watch()

In [70]:
# eqd.sample.watch()

### Purcell and resonator for qubit3

In [71]:
# Add coupler purcell + resonator with fanout and grounding
pur_res_coupler3_position = [(eqd.coaxmon4.center[0]+100 + purcell_resonator_coupler_length, eqd.coaxmon4.center[1]-450),
                             (eqd.coaxmon4.center[0]+100,
                              eqd.coaxmon4.center[1]-450)]
pur_res_coupler3 = elements.CPWCoupler('purcell_resonator_coupler3',
                                         pur_res_coupler3_position,
                                         pur_res_coupler_w,
                                         pur_res_coupler_s,
                                         eqd.resonator_ground,
                                         eqd.sample.layer_configuration,
                                         r=0)
eqd.sample.add(pur_res_coupler3)

pur_res_coupler3_fanout = eqd.sample.fanout(o=pur_res_coupler3,
                                            port='port1',
                                            name='res-purcell-coupler-fanout',
                                            grouping=[1, 3])
pur_res_coupler3_ground = eqd.sample.ground(
    o=pur_res_coupler3,
    port='port2',
    name='res-purcell-coupler-grounding',
    grounding_width=res_pur_coupler_grounding,
    grounding_between=[(0, 4)])

pur_res_coupler3_fanout_ground = eqd.sample.ground(
    o=pur_res_coupler3_fanout,
    port='center',
    name='res-purcell-coupler-fanout-grounding',
    grounding_width=res_pur_coupler_grounding,
    grounding_between=[(2, 3)])

In [72]:
# Add resonator for qubit 3
name = 'resonator3'
begin_point = (pur_res_coupler3_fanout.terminals['up'].position[0],
               pur_res_coupler3_fanout.terminals['up'].position[1]+10)
begin_orientation = np.pi / 2

end_point = (eqd.coaxmon3.terminals['coupler1'].position[0]-100,
             eqd.coaxmon3.terminals['coupler1'].position[1]-100)
end_orientation =  - np.pi / 2
meander_length = 3700
left_limit = 0
right_limit = 330
radius_limit = (45,80)
w = eqd.resonator_core
s = eqd.resonator_gap
g = eqd.resonator_ground
result1, points1, real_points1, resonator3 = meander.meander_creation2(name, begin_point, begin_orientation,
                                    end_point, end_orientation, meander_length,
                                    left_limit, right_limit, radius_limit, w,
                                    s, g, eqd.sample.layer_configuration)

eqd.sample.add(resonator3)
eqd.sample.generate_bridge_over_cpw(name='resonator3_airbridges', o=resonator3, geometry=airbridge,
                                 min_spacing=resonator_min_bridge_spacing, bridge_part_decimation=2)

connection = eqd.sample.connect_cpw(resonator3,
                           pur_res_coupler3_fanout,
                           'port1',
                           'up',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=80)
print(connection[0].length)

A_max, B_max 235.42282572551983 68.23221769825102
489.77572326877527
A, K, B =  235.4228257255198 326.9663312960441 68.232217698251
A_max, B_max 256.31711929413984 89.12651126687106
408.03414044476426
A, K, B =  256.3171192941398 245.22474847203304 89.12651126687105
A_max, B_max 268.8536954353119 101.66308740804308
358.9891907503576
A, K, B =  262.69249255768045 196.17979877762642 96.29669819267707
points [(2611.0, 2852.0), (2611.0, 3164.8387971223683), (2711.292609129376, 3164.8387971223683), (2711.292609129376, 2868.366389215366), (2811.5852182587523, 2868.366389215366), (2811.5852182587523, 3164.8387971223683), (2911.8778273881285, 3164.8387971223683), (2911.8778273881285, 2868.366389215366), (3012.1704365175046, 2868.366389215366), (3012.1704365175046, 3164.8387971223683), (3112.4630456468813, 3164.8387971223683), (3112.4630456468813, 2868.366389215366), (3212.755654776257, 2868.366389215366), (3212.755654776257, 3164.8387971223683), (3313.0482639056336, 3164.8387971223683), (3313.

In [73]:
# Add purcell closed end
name = 'purcell_closed_end3'
begin_point = (TL_purcell_couplers[3][0].terminals['up'].position[0] - 100,
               TL_purcell_couplers[3][0].terminals['up'].position[1] + 100)
begin_orientation = np.pi
end_point = (pur_res_coupler3_fanout_ground.terminals['narrow'].position[0]+500,
            pur_res_coupler3_fanout_ground.terminals['narrow'].position[1])
end_orientation =   0
meander_length = 2000
left_limit = 500
right_limit = 500
radius_limit = (40,90)
w = eqd.resonator_core
s = eqd.resonator_gap
g = eqd.resonator_ground

result1, points1, real_points1, purcell_closed_end_3 = meander.meander_creation2(name, begin_point, begin_orientation,
                                    end_point, end_orientation, meander_length,
                                    left_limit, right_limit, radius_limit, w,
                                    s, g, eqd.sample.layer_configuration)
eqd.sample.add(purcell_closed_end_3)
eqd.sample.generate_bridge_over_cpw(name='purcell_closed_end_3_airbridges', o=purcell_closed_end_3, geometry=airbridge,
                                 min_spacing=resonator_min_bridge_spacing, bridge_part_decimation=2)

A_max, B_max 416.0 777.0
951.1637362879451
A, K, B =  376.08571640741656 590.1637362879451 575.0780198805285
points [(3493.0, 2487.0), (3043.9142835925836, 2487.0), (3043.9142835925836, 2633.0), (3780.0780198805287, 2633.0), (3780.0780198805287, 2779.0), (3132.0, 2779.0)] real points [(3493.0, 2487.0), (3116.9142835925836, 2487.0), (3116.9142835925836, 2633.0), (3707.0780198805287, 2633.0), (3707.0780198805287, 2779.0), (3132.0, 2779.0)] 3.141592653589793 0


[CPW "purcell_closed_end_3_airbridgess0", l=29.011,
 CPW "purcell_closed_end_3_airbridgess1", l=36.000,
 CPW "purcell_closed_end_3_airbridgess1", l=58.021,
 CPW "purcell_closed_end_3_airbridgess3", l=58.021,
 CPW "purcell_closed_end_3_airbridgess4", l=36.000,
 CPW "purcell_closed_end_3_airbridgess4", l=58.021,
 CPW "purcell_closed_end_3_airbridgess6", l=289.527,
 CPW "purcell_closed_end_3_airbridgess7", l=36.000,
 CPW "purcell_closed_end_3_airbridgess7", l=62.361,
 CPW "purcell_closed_end_3_airbridgess9", l=62.361,
 CPW "purcell_closed_end_3_airbridgess10", l=36.000,
 CPW "purcell_closed_end_3_airbridgess10", l=62.361,
 CPW "purcell_closed_end_3_airbridgess12", l=62.361,
 CPW "purcell_closed_end_3_airbridgess13", l=36.000,
 CPW "purcell_closed_end_3_airbridgess13", l=62.361,
 CPW "purcell_closed_end_3_airbridgess15", l=290.440,
 CPW "purcell_closed_end_3_airbridgess16", l=36.000,
 CPW "purcell_closed_end_3_airbridgess16", l=59.846,
 CPW "purcell_closed_end_3_airbridgess18", l=59.846,
 

In [74]:
# Add purcell open end
name = 'purcell_open_end3'
begin_point = (TL_purcell_couplers[3][1].terminals['down'].position[0],
               TL_purcell_couplers[3][1].terminals['down'].position[1]+10)
begin_orientation = np.pi / 2
end_point = (2650, 2600)
end_orientation =   - np.pi / 2
meander_length = 1000
left_limit = 0
right_limit = 320
radius_limit = (40,90)
w = eqd.resonator_core
s = eqd.resonator_gap
g = eqd.resonator_ground

result1, points1, real_points1, purcell_open_end3 = meander.meander_creation2(name, begin_point, begin_orientation,
                                    end_point, end_orientation, meander_length,
                                    left_limit, right_limit, radius_limit, w,
                                    s, g, eqd.sample.layer_configuration)
eqd.sample.add(purcell_open_end3)
eqd.sample.generate_bridge_over_cpw(name='purcell_open_end3_airbridges', o=purcell_open_end3, geometry=airbridge,
                                 min_spacing=resonator_min_bridge_spacing, bridge_part_decimation=1)

connection = eqd.sample.connect_cpw(purcell_open_end3,
                           TL_purcell_couplers[3][1],
                           'port1',
                           'down',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=80)

print(connection[0].length)

# Add open end for purcell 
eqd.sample.open_end(purcell_open_end3, 'port2', 'purcell3-open-end')

A_max, B_max 228.75 111.75
349.3871895494191
A, K, B =  228.74999999999997 146.3871895494191 111.74999999999999
A_max, B_max 268.875 151.875
276.19359477470954
A, K, B =  138.09679738735477 73.19359477470955 138.09679738735477
points [(2971.0, 2397.0), (2971.0, 2575.221797387355), (2890.75, 2575.221797387355), (2890.75, 2421.778202612645), (2810.5, 2421.778202612645), (2810.5, 2575.221797387355), (2730.25, 2575.221797387355), (2730.25, 2421.778202612645), (2650.0, 2421.778202612645), (2650, 2600)] real points [(2971.0, 2397.0), (2971.0, 2535.096797387355), (2890.75, 2535.096797387355), (2890.75, 2461.903202612645), (2810.5, 2461.903202612645), (2810.5, 2535.096797387355), (2730.25, 2535.096797387355), (2730.25, 2461.903202612645), (2650.0, 2461.903202612645), (2650, 2600)] 1.5707963267948966 -1.5707963267948966
10.0


OpenEnd purcell3-open-end

In [75]:
connection = eqd.sample.connect_cpw(TL_purcell_couplers[3][0],
                           purcell_closed_end_3,
                           'up',
                           'port1',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=80)
print(connection[0].length)

connection = eqd.sample.connect_cpw(pur_res_coupler3_fanout_ground,
                           purcell_closed_end_3,
                           'narrow',
                           'port2',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing / 5,
                           r=80)
print(connection[0].length)

connection = eqd.sample.connect_cpw(eqd.coaxmon3,
                           resonator3,
                           'coupler1',
                           'port2',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=80)

print(connection[0].length)

164.1807592322446
65.33333333333348
151.30577095386928


In [76]:
eqd.sample.draw_design()
# eqd.sample.watch()

In [77]:
eqd.sample.watch()

### Purcell and resonator for qubit7

In [78]:
# Add coupler purcell + resonator with fanout and grounding
pur_res_coupler7_position = [(eqd.coaxmon8.center[0] + 470, eqd.coaxmon8.center[1]-445),
                             (eqd.coaxmon8.center[0] + 470 - purcell_resonator_coupler_length, eqd.coaxmon8.center[1]-445)]
pur_res_coupler7 = elements.CPWCoupler('purcell_resonator_coupler7',
                                         pur_res_coupler7_position,
                                         pur_res_coupler_w,
                                         pur_res_coupler_s,
                                         eqd.resonator_ground,
                                         eqd.sample.layer_configuration,
                                         r=0)
eqd.sample.add(pur_res_coupler7)

pur_res_coupler7_fanout = eqd.sample.fanout(o=pur_res_coupler7,
                                            port='port2',
                                            name='res-purcell-coupler-fanout',
                                            grouping=[1, 3])
pur_res_coupler7_ground = eqd.sample.ground(
    o=pur_res_coupler7,
    port='port1',
    name='res-purcell-coupler-grounding',
    grounding_width=res_pur_coupler_grounding,
    grounding_between=[(0, 4)])

pur_res_coupler7_fanout_ground = eqd.sample.ground(
    o=pur_res_coupler7_fanout,
    port='center',
    name='res-purcell-coupler-fanout-grounding',
    grounding_width=res_pur_coupler_grounding,
    grounding_between=[(0, 1)])

In [79]:
# Add resonator for qubit 7
name = 'resonator7'
begin_point = (pur_res_coupler7_fanout.terminals['down'].position[0],
               pur_res_coupler7_fanout.terminals['down'].position[1]+10)
begin_orientation = np.pi / 2
end_point = (eqd.coaxmon7.terminals['coupler1'].position[0]-100,
             eqd.coaxmon7.terminals['coupler1'].position[1]-100)
end_orientation =   - np.pi / 2
meander_length = 3100
left_limit = 0
right_limit = 240
radius_limit = (40,80)
w = eqd.resonator_core
s = eqd.resonator_gap
g = eqd.resonator_ground
result1, points1, real_points1, resonator7 = meander.meander_creation2(name, begin_point, begin_orientation,
                                    end_point, end_orientation, meander_length,
                                    left_limit, right_limit, radius_limit, w,
                                    s, g, eqd.sample.layer_configuration)

eqd.sample.add(resonator7)
eqd.sample.generate_bridge_over_cpw(name='resonator7_airbridges', o=resonator7, geometry=airbridge,
                                 min_spacing=resonator_min_bridge_spacing, bridge_part_decimation=2)

connection = eqd.sample.connect_cpw(resonator7,
                           pur_res_coupler7_fanout,
                           'port1',
                           'down',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=80)
print(connection[0].length)

connection = eqd.sample.connect_cpw(eqd.coaxmon7,
                           resonator7,
                           'coupler1',
                           'port2',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=80)
print(connection[0].length)

A_max, B_max 141.08949239218646 58.89888436491765
371.99548843655265
A, K, B =  141.08949239218643 214.18609646382146 58.89888436491764
A_max, B_max 163.06711929413984 80.87651126687103
318.44896432059727
A, K, B =  163.06711929413981 160.6395723478661 80.87651126687102
A_max, B_max 176.25369543531187 94.06308740804306
286.3210498510241
A, K, B =  176.25369543531184 128.51165787829288 94.06308740804305
A_max, B_max 185.04474619609323 102.85413816882442
264.90244020464195
A, K, B =  170.32938984289325 107.09304823191073 94.57305036174866
points [(5759.0, 2857.0), (5759.0, 3071.2846436468), (5846.910507607814, 3071.2846436468), (5846.910507607814, 2876.2810878070754), (5934.821015215627, 2876.2810878070754), (5934.821015215627, 3071.2846436468), (6022.73152282344, 3071.2846436468), (6022.73152282344, 2876.2810878070754), (6110.642030431254, 2876.2810878070754), (6110.642030431254, 3071.2846436468), (6198.552538039068, 3071.2846436468), (6198.552538039068, 2876.2810878070754), (6286.46304

In [80]:
# Add purcell closed end 7
name = 'purcell_closed_end7'
begin_point = (TL_purcell_couplers[5][1].terminals['down'].position[0] - 100,
               TL_purcell_couplers[5][1].terminals['down'].position[1] + 100)
begin_orientation = np.pi
end_point = (pur_res_coupler7_fanout_ground.terminals['narrow'].position[0],
             pur_res_coupler7_fanout_ground.terminals['narrow'].position[1]-80)
end_orientation =   0
meander_length = 2000
left_limit = 480
right_limit = 350
radius_limit = (40,90)
w = eqd.resonator_core
s = eqd.resonator_gap
g = eqd.resonator_ground

result1, points1, real_points1, purcell_closed_end_7 = meander.meander_creation2(name, begin_point, begin_orientation,
                                    end_point, end_orientation, meander_length,
                                    left_limit, right_limit, radius_limit, w,
                                    s, g, eqd.sample.layer_configuration)
eqd.sample.add(purcell_closed_end_7)
eqd.sample.generate_bridge_over_cpw(name='purcell_closed_end_7_airbridges', o=purcell_closed_end_7, geometry=airbridge,
                                 min_spacing=resonator_min_bridge_spacing, bridge_part_decimation=2)

A_max, B_max 414.75 617.75
996.0685985427538
A, K, B =  399.8033805820643 663.0685985427538 596.2652179606896
points [(6071.0, 2487.0), (5616.946619417936, 2487.0), (5616.946619417936, 2595.5), (6388.515217960689, 2595.5), (6388.515217960689, 2704.0), (5738.0, 2704.0)] real points [(6071.0, 2487.0), (5671.196619417936, 2487.0), (5671.196619417936, 2595.5), (6334.265217960689, 2595.5), (6334.265217960689, 2704.0), (5738.0, 2704.0)] 3.141592653589793 0


[CPW "purcell_closed_end_7_airbridgess0", l=31.975,
 CPW "purcell_closed_end_7_airbridgess1", l=36.000,
 CPW "purcell_closed_end_7_airbridgess1", l=63.951,
 CPW "purcell_closed_end_7_airbridgess3", l=63.951,
 CPW "purcell_closed_end_7_airbridgess4", l=36.000,
 CPW "purcell_closed_end_7_airbridgess4", l=63.951,
 CPW "purcell_closed_end_7_airbridgess6", l=231.769,
 CPW "purcell_closed_end_7_airbridgess7", l=36.000,
 CPW "purcell_closed_end_7_airbridgess7", l=58.724,
 CPW "purcell_closed_end_7_airbridgess9", l=58.724,
 CPW "purcell_closed_end_7_airbridgess10", l=36.000,
 CPW "purcell_closed_end_7_airbridgess10", l=58.724,
 CPW "purcell_closed_end_7_airbridgess12", l=58.724,
 CPW "purcell_closed_end_7_airbridgess13", l=36.000,
 CPW "purcell_closed_end_7_airbridgess13", l=58.724,
 CPW "purcell_closed_end_7_airbridgess15", l=58.724,
 CPW "purcell_closed_end_7_airbridgess16", l=36.000,
 CPW "purcell_closed_end_7_airbridgess16", l=231.482,
 CPW "purcell_closed_end_7_airbridgess18", l=63.378,
 

In [81]:
# Add purcell open end
name = 'purcell_open_end7'
begin_point = (TL_purcell_couplers[5][0].terminals['up'].position[0],
               TL_purcell_couplers[5][0].terminals['up'].position[1]+10)
begin_orientation = np.pi / 2
end_point = (6470, 2655)
end_orientation =   - np.pi / 2
meander_length = 1400
left_limit = 0
right_limit = 430
radius_limit = (40,90)
w = eqd.resonator_core
s = eqd.resonator_gap
g = eqd.resonator_ground

result1, points1, real_points1, purcell_open_end7 = meander.meander_creation2(name, begin_point, begin_orientation,
                                    end_point, end_orientation, meander_length,
                                    left_limit, right_limit, radius_limit, w,
                                    s, g, eqd.sample.layer_configuration)
eqd.sample.add(purcell_open_end7)
eqd.sample.generate_bridge_over_cpw(name='purcell_open_end7_airbridges', o=purcell_open_end7, geometry=airbridge,
                                 min_spacing=resonator_min_bridge_spacing, bridge_part_decimation=2)


connection = eqd.sample.connect_cpw(purcell_open_end7,
                           TL_purcell_couplers[5][0],
                           'port1',
                           'up',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=80)

print(connection[0].length)

A_max, B_max 338.25 166.25
575.3163932226242
A, K, B =  338.24999999999994 317.31639322262424 166.24999999999997
A_max, B_max 378.625 206.625
416.6581966113121
A, K, B =  228.2755368624345 158.65819661131212 188.38265974887742
points [(6793.0, 2397.0), (6793.0, 2665.6505368624344), (6712.25, 2665.6505368624344), (6712.25, 2426.242340251122), (6631.5, 2426.242340251122), (6631.5, 2665.6505368624344), (6550.75, 2665.6505368624344), (6550.75, 2426.242340251122), (6470.0, 2426.242340251122), (6470, 2655)] real points [(6793.0, 2397.0), (6793.0, 2625.2755368624344), (6712.25, 2625.2755368624344), (6712.25, 2466.617340251122), (6631.5, 2466.617340251122), (6631.5, 2625.2755368624344), (6550.75, 2625.2755368624344), (6550.75, 2466.617340251122), (6470.0, 2466.6173402511226), (6470, 2655)] 1.5707963267948966 -1.5707963267948966
10.0


In [82]:
connection = eqd.sample.connect_cpw(purcell_closed_end_7,
                           pur_res_coupler7_fanout_ground,
                           'port2',
                           'narrow',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=40)
print(connection[0].length)

209.82770192589226


In [83]:
connection = eqd.sample.connect_cpw(TL_purcell_couplers[5][1],
                           purcell_closed_end_7,
                           'down',
                           'port1',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=80)
print(connection[0].length)

164.18075923224342


In [84]:
# Add open end for purcell 
eqd.sample.open_end(purcell_open_end7, 'port2', 'purcell7-open-end')

OpenEnd purcell7-open-end

In [85]:
eqd.sample.draw_design()
# eqd.sample.watch()

### Purcell and resonator for qubit5 

In [86]:
# Add coupler purcell + resonator with fanout and grounding
pur_res_coupler5_position = [(eqd.coaxmon6.center[0]-100 - purcell_resonator_coupler_length,
                              eqd.coaxmon6.center[1]+450),
                             (eqd.coaxmon6.center[0]-100,
                              eqd.coaxmon6.center[1]+450)]
pur_res_coupler5 = elements.CPWCoupler('purcell_resonator_coupler5',
                                         pur_res_coupler5_position,
                                         pur_res_coupler_w,
                                         pur_res_coupler_s,
                                         eqd.resonator_ground,
                                         eqd.sample.layer_configuration,
                                         r=0)
eqd.sample.add(pur_res_coupler5)

pur_res_coupler5_fanout = eqd.sample.fanout(o=pur_res_coupler5,
                                            port='port1',
                                            name='res-purcell-coupler-fanout',
                                            grouping=[1, 3])
pur_res_coupler5_ground = eqd.sample.ground(
    o=pur_res_coupler5,
    port='port2',
    name='res-purcell-coupler-grounding',
    grounding_width=res_pur_coupler_grounding,
    grounding_between=[(0, 4)])

pur_res_coupler5_fanout_ground = eqd.sample.ground(
    o=pur_res_coupler5_fanout,
    port='center',
    name='res-purcell-coupler-fanout-grounding',
    grounding_width=res_pur_coupler_grounding,
    grounding_between=[(2, 3)])

In [87]:
# Add resonator for qubit 5
name = 'resonator5'
begin_point = (pur_res_coupler5_fanout.terminals['up'].position[0],
               pur_res_coupler5_fanout.terminals['up'].position[1]-10)
begin_orientation = - np.pi / 2

end_point = (eqd.coaxmon5.terminals['coupler1'].position[0]+100,
             eqd.coaxmon5.terminals['coupler1'].position[1]+100)
end_orientation =   np.pi / 2
meander_length = 3700
left_limit = 330
right_limit = 0
radius_limit = (45,80)
w = eqd.resonator_core
s = eqd.resonator_gap
g = eqd.resonator_ground
result1, points1, real_points1, resonator5 = meander.meander_creation2(name, begin_point, begin_orientation,
                                    end_point, end_orientation, meander_length,
                                    left_limit, right_limit, radius_limit, w,
                                    s, g, eqd.sample.layer_configuration)

eqd.sample.add(resonator5)
eqd.sample.generate_bridge_over_cpw(name='resonator5_airbridges', o=resonator5, geometry=airbridge,
                                 min_spacing=resonator_min_bridge_spacing, bridge_part_decimation=2)

connection = eqd.sample.connect_cpw(resonator5,
                           pur_res_coupler5_fanout,
                           'port1',
                           'up',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=80)
print(connection[0].length)

A_max, B_max 235.42282572551977 68.23221769825098
489.7757232687751
A, K, B =  235.42282572551974 326.9663312960439 68.23221769825096
A_max, B_max 256.31711929413984 89.12651126687103
408.03414044476415
A, K, B =  256.3171192941398 245.22474847203296 89.12651126687102
A_max, B_max 268.85369543531186 101.66308740804307
358.9891907503575
A, K, B =  262.69249255768045 196.17979877762633 96.2966981926771
points [(6789.0, 1848.0), (6789.0, 1535.1612028776315), (6688.707390870623, 1535.1612028776315), (6688.707390870623, 1831.6336107846341), (6588.414781741248, 1831.6336107846341), (6588.414781741248, 1535.1612028776315), (6488.122172611871, 1535.1612028776315), (6488.122172611871, 1831.6336107846341), (6387.829563482495, 1831.6336107846341), (6387.829563482495, 1535.1612028776315), (6287.536954353119, 1535.1612028776315), (6287.536954353119, 1831.6336107846341), (6187.244345223742, 1831.6336107846341), (6187.244345223742, 1535.1612028776315), (6086.951736094366, 1535.1612028776315), (6086.9

In [88]:
# Add purcell closed end
name = 'purcell_closed_end5'
begin_point = (TL_purcell_couplers[4][0].terminals['up'].position[0] + 100,
               TL_purcell_couplers[4][0].terminals['up'].position[1] - 100)
begin_orientation = 0
end_point = (pur_res_coupler5_fanout_ground.terminals['narrow'].position[0]-500,
            pur_res_coupler5_fanout_ground.terminals['narrow'].position[1])
end_orientation =   np.pi
meander_length = 2000
left_limit = 500
right_limit = 500
radius_limit = (40,90)
w = eqd.resonator_core
s = eqd.resonator_gap
g = eqd.resonator_ground

result1, points1, real_points1, purcell_closed_end_5 = meander.meander_creation2(name, begin_point, begin_orientation,
                                    end_point, end_orientation, meander_length,
                                    left_limit, right_limit, radius_limit, w,
                                    s, g, eqd.sample.layer_configuration)
eqd.sample.add(purcell_closed_end_5)
eqd.sample.generate_bridge_over_cpw(name='purcell_closed_end_5_airbridges', o=purcell_closed_end_5, geometry=airbridge,
                                 min_spacing=resonator_min_bridge_spacing, bridge_part_decimation=2)

A_max, B_max 416.0 777.0
951.1637362879451
A, K, B =  376.08571640741656 590.1637362879451 575.0780198805285
points [(5907.0, 2213.0), (6356.085716407417, 2213.0), (6356.085716407417, 2067.0), (5619.921980119472, 2067.0), (5619.921980119472, 1921.0), (6268.0, 1921.0)] real points [(5907.0, 2213.0), (6283.085716407417, 2213.0), (6283.085716407417, 2067.0), (5692.921980119472, 2067.0), (5692.921980119471, 1921.0), (6268.0, 1921.0)] 0 3.141592653589793


[CPW "purcell_closed_end_5_airbridgess0", l=29.011,
 CPW "purcell_closed_end_5_airbridgess1", l=36.000,
 CPW "purcell_closed_end_5_airbridgess1", l=58.021,
 CPW "purcell_closed_end_5_airbridgess3", l=58.021,
 CPW "purcell_closed_end_5_airbridgess4", l=36.000,
 CPW "purcell_closed_end_5_airbridgess4", l=58.021,
 CPW "purcell_closed_end_5_airbridgess6", l=289.527,
 CPW "purcell_closed_end_5_airbridgess7", l=36.000,
 CPW "purcell_closed_end_5_airbridgess7", l=62.361,
 CPW "purcell_closed_end_5_airbridgess9", l=62.361,
 CPW "purcell_closed_end_5_airbridgess10", l=36.000,
 CPW "purcell_closed_end_5_airbridgess10", l=62.361,
 CPW "purcell_closed_end_5_airbridgess12", l=62.361,
 CPW "purcell_closed_end_5_airbridgess13", l=36.000,
 CPW "purcell_closed_end_5_airbridgess13", l=62.361,
 CPW "purcell_closed_end_5_airbridgess15", l=290.440,
 CPW "purcell_closed_end_5_airbridgess16", l=36.000,
 CPW "purcell_closed_end_5_airbridgess16", l=59.846,
 CPW "purcell_closed_end_5_airbridgess18", l=59.846,
 

In [89]:
# Add purcell open end
name = 'purcell_open_end5'
begin_point = (TL_purcell_couplers[4][1].terminals['down'].position[0],
               TL_purcell_couplers[4][1].terminals['down'].position[1]-10)
begin_orientation = - np.pi / 2
end_point = (6750, 2175)
end_orientation =   np.pi / 2
meander_length = 1000
left_limit = 320
right_limit = 0
radius_limit = (40,90)
w = eqd.resonator_core
s = eqd.resonator_gap
g = eqd.resonator_ground

result1, points1, real_points1, purcell_open_end5 = meander.meander_creation2(name, begin_point, begin_orientation,
                                    end_point, end_orientation, meander_length,
                                    left_limit, right_limit, radius_limit, w,
                                    s, g, eqd.sample.layer_configuration)
eqd.sample.add(purcell_open_end5)
eqd.sample.generate_bridge_over_cpw(name='purcell_open_end5_airbridges', o=purcell_open_end5, geometry=airbridge,
                                 min_spacing=resonator_min_bridge_spacing, bridge_part_decimation=2)



connection = eqd.sample.connect_cpw(purcell_open_end5,
                           TL_purcell_couplers[4][1],
                           'port1',
                           'down',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=80)

# Add open end for purcell 
eqd.sample.open_end(purcell_open_end5, 'port2', 'purcell3-open-end')

A_max, B_max 228.75 36.75
311.8871895494191
A, K, B =  228.74999999999997 183.8871895494191 36.74999999999999
A_max, B_max 268.875 76.875
219.94359477470954
A, K, B =  164.75499635685983 91.94359477470955 55.18859841784951
points [(6429.0, 2303.0), (6429.0, 2098.12000364314), (6509.25, 2098.12000364314), (6509.25, 2270.31359841785), (6589.5, 2270.31359841785), (6589.5, 2098.12000364314), (6669.75, 2098.12000364314), (6669.75, 2270.31359841785), (6750.0, 2270.31359841785), (6750, 2175)] real points [(6429.0, 2303.0), (6429.0, 2138.24500364314), (6509.25, 2138.24500364314), (6509.25, 2230.18859841785), (6589.5, 2230.18859841785), (6589.5, 2138.24500364314), (6669.75, 2138.24500364314), (6669.75, 2230.18859841785), (6750.0, 2230.1885984178493), (6750, 2175)] -1.5707963267948966 1.5707963267948966


OpenEnd purcell3-open-end

In [90]:
connection = eqd.sample.connect_cpw(TL_purcell_couplers[4][0],
                           purcell_closed_end_5,
                           'up',
                           'port1',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=80)
print(connection[0].length)

connection = eqd.sample.connect_cpw(pur_res_coupler5_fanout_ground,
                           purcell_closed_end_5,
                           'narrow',
                           'port2',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing/5,
                           r=80)
print(connection[0].length)

connection = eqd.sample.connect_cpw(eqd.coaxmon5,
                           resonator5,
                           'coupler1',
                           'port2',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=80)

print(connection[0].length)


164.18075923224342
65.33333333333303
151.30577095386911


In [91]:
eqd.sample.draw_design()
# eqd.sample.watch()

### Purcell and resonator for qubit4 

In [92]:
# Add open end purcell meander with open ending
name = 'purcell_open_end4'
begin_point = (TL_purcell_couplers[0][0].terminals['down'].position[0],
               TL_purcell_couplers[0][0].terminals['down'].position[1]+10)
begin_orientation = np.pi / 2
end_point = (560, 3430)
end_orientation =   -np.pi / 2
meander_length = 3950
left_limit = 10
right_limit = 1070
radius_limit = (40,90)
w = eqd.resonator_core
s = eqd.resonator_gap
g = eqd.resonator_ground

result1, points1, real_points1, purcell_open_end4 = meander.meander_creation2(name, begin_point, begin_orientation,
                                    end_point, end_orientation, meander_length,
                                    left_limit, right_limit, radius_limit, w,
                                    s, g, eqd.sample.layer_configuration)
eqd.sample.add(purcell_open_end4)
eqd.sample.generate_bridge_over_cpw(name='purcell_open_end4_airbridges', o=purcell_open_end4, geometry=airbridge,
                                 min_spacing=resonator_min_bridge_spacing, bridge_part_decimation=2)



connection = eqd.sample.connect_cpw(purcell_open_end4,
                           TL_purcell_couplers[0][0],
                           'port1',
                           'down',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=80)

# Add open end for purcell 
eqd.sample.open_end(purcell_open_end4, 'port2', 'purcell4-open-end')

A_max, B_max 951.75 924.75
2154.5641879024947
A, K, B =  951.7499999999999 1121.5641879024947 924.7499999999999
A_max, B_max 1005.375 978.375
1593.7820939512474
A, K, B =  796.8910469756237 560.7820939512474 796.8910469756237
points [(989.0, 2397.0), (989.0, 3247.516046975624), (881.75, 3247.516046975624), (881.75, 2579.483953024376), (774.5, 2579.483953024376), (774.5, 3247.516046975624), (667.25, 3247.516046975624), (667.25, 2579.483953024376), (560.0, 2579.483953024376), (560, 3430)] real points [(989.0, 2397.0), (989.0, 3193.891046975624), (881.75, 3193.891046975624), (881.75, 2633.108953024376), (774.5, 2633.108953024376), (774.5, 3193.891046975624), (667.25, 3193.891046975624), (667.25, 2633.108953024376), (560.0, 2633.108953024376), (560, 3430)] 1.5707963267948966 -1.5707963267948966


OpenEnd purcell4-open-end

In [93]:
# Add coupler purcell + resonator with fanout and grounding
pur_res_coupler4_position = [
    (TL_purcell_couplers[0][1].terminals['up'].position[0] - 
     purcell_resonator_coupler_length,
     TL_purcell_couplers[0][1].terminals['up'].position[1] + 225),
    (TL_purcell_couplers[0][1].terminals['up'].position[0],
     TL_purcell_couplers[0][1].terminals['up'].position[1] + 225)
]
pur_res_coupler4 = elements.CPWCoupler('purcell_resonator_coupler4',
                                       pur_res_coupler4_position,
                                       pur_res_coupler_w,
                                       pur_res_coupler_s,
                                       eqd.resonator_ground,
                                       eqd.sample.layer_configuration,
                                       r=0)
eqd.sample.add(pur_res_coupler4)

pur_res_coupler4_fanout = eqd.sample.fanout(o=pur_res_coupler4,
                                            port='port1',
                                            name='res-purcell-coupler-fanout',
                                            grouping=[0, 2])
pur_res_coupler4_ground = eqd.sample.ground(
    o=pur_res_coupler4,
    port='port2',
    name='res-purcell-coupler-grounding',
    grounding_width=res_pur_coupler_grounding,
    grounding_between=[(0, 4)])

pur_res_coupler4_fanout_ground = eqd.sample.ground(
    o=pur_res_coupler4_fanout,
    port='center',
    name='res-purcell-coupler-fanout-grounding',
    grounding_width=res_pur_coupler_grounding,
    grounding_between=[(0, 1)])

In [94]:
# Add closed end purcell
name = 'purcell_closed_end4'
begin_point = (pur_res_coupler4_fanout_ground.terminals['narrow'].position[0]-10,
               pur_res_coupler4_fanout_ground.terminals['narrow'].position[1])
begin_orientation = - np.pi 
end_point = (TL_purcell_couplers[0][1].terminals['up'].position[0]-100,
            TL_purcell_couplers[0][1].terminals['up'].position[1]+100)
end_orientation =   np.pi
meander_length = 600
left_limit = 500
right_limit = 100
radius_limit = (40,90)
w = eqd.resonator_core
s = eqd.resonator_gap
g = eqd.resonator_ground

result1, points1, real_points1, purcell_closed_end4 = meander.meander_creation2(name, begin_point, begin_orientation,
                                    end_point, end_orientation, meander_length,
                                    left_limit, right_limit, radius_limit, w,
                                    s, g, eqd.sample.layer_configuration)
eqd.sample.add(purcell_closed_end4)
eqd.sample.generate_bridge_over_cpw(name='purcell_closed_end4_airbridges', o=purcell_closed_end4, geometry=airbridge,
                                 min_spacing=resonator_min_bridge_spacing, bridge_part_decimation=2)

connection = eqd.sample.connect_cpw(purcell_closed_end4,
                           pur_res_coupler4_fanout_ground,
                           'port1',
                           'narrow',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=80)

print(connection[0].length)

A_max, B_max 437.0 579.0
A, K, B =  218.31859100666537 0 218.31859100666537
points [(1369.0, 2591.0), (1098.6814089933346, 2591.0), (1098.6814089933346, 2487.0), (1511.0, 2487.0)] real points [(1369.0, 2591.0), (1150.6814089933346, 2591.0), (1292.6814089933346, 2487.0), (1511.0, 2487.0)] -3.141592653589793 3.141592653589793
10.0


In [95]:
connection = eqd.sample.connect_cpw(TL_purcell_couplers[0][1],
                           purcell_closed_end4,
                           'up',
                           'port2',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=80)
print(connection[0].length)

164.18075923224467


In [96]:
# Add resonator
name = 'resonator4'
begin_point = (pur_res_coupler4_fanout.terminals['down'].position[0],
               pur_res_coupler4_fanout.terminals['down'].position[1]+10)
begin_orientation = np.pi  / 2
end_point = (eqd.coaxmon4.terminals['coupler1'].position[0]- 60,
            eqd.coaxmon4.terminals['coupler1'].position[1]-100)
end_orientation =   - np.pi / 2
meander_length = 3700
left_limit = 10
right_limit = 810
radius_limit = (40,90)
w = eqd.resonator_core
s = eqd.resonator_gap
g = eqd.resonator_ground

result1, points1, real_points1, resonator4 = meander.meander_creation2(name, begin_point, begin_orientation,
                                    end_point, end_orientation, meander_length,
                                    left_limit, right_limit, radius_limit, w,
                                    s, g, eqd.sample.layer_configuration)
eqd.sample.add(resonator4)
eqd.sample.generate_bridge_over_cpw(name='resonator4_airbridges', o=resonator4, geometry=airbridge,
                                 min_spacing=resonator_min_bridge_spacing, bridge_part_decimation=2)

connection = eqd.sample.connect_cpw(resonator4,
                           pur_res_coupler4_fanout,
                           'port1',
                           'down',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=0)

print(connection[0].length)

A_max, B_max 717.2592385882797 268.06863056101093
931.3108684296519
A, K, B =  687.002856021439 580.5014764569207 244.3080124082125
A_max, B_max 744.5061590588532 295.3155510315844
737.8103762773451
A, K, B =  491.97249477037343 387.00098430461384 245.83788150697157
A_max, B_max 758.1296192941398 308.93901126687103
641.0601302011916
A, K, B =  360.2134313023737 290.25073822846036 280.8466988988178
points [(1400.0, 2664.0), (1400.0, 3432.743617433159), (1563.4815228234406, 3432.743617433159), (1563.4815228234406, 2688.760618152798), (1726.963045646881, 2688.760618152798), (1726.963045646881, 3432.743617433159), (1890.4445684703214, 3432.743617433159), (1890.4445684703214, 2688.760618152798), (2053.926091293762, 2688.760618152798), (2053.926091293762, 3014.809391972731)] real points [(1400.0, 2664.0), (1400.0, 3351.002856021439), (1563.4815228234406, 3351.002856021439), (1563.4815228234406, 2770.501379564518), (1726.963045646881, 2770.501379564518), (1726.963045646881, 3351.002856021439)

In [97]:
connection = eqd.sample.connect_cpw(resonator4,
                           eqd.coaxmon4,
                           'port2',
                           'coupler1',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=100)
print(connection[0].length)

122.14889754564788


In [98]:
eqd.sample.draw_design()
# eqd.sample.watch()

In [99]:
# eqd.sample.watch()

### Purcell and resonator for qubit 6 

In [100]:
# Add open end purcell meander with open ending
name = 'purcell_open_end6'
begin_point = (TL_purcell_couplers[1][0].terminals['down'].position[0],
               TL_purcell_couplers[1][0].terminals['down'].position[1]-10)
begin_orientation = - np.pi / 2
end_point = (8830, 1245)
end_orientation =   np.pi / 2
meander_length = 3800
left_limit = 1070
right_limit = 10
radius_limit = (40,90)
w = eqd.resonator_core
s = eqd.resonator_gap
g = eqd.resonator_ground

result1, points1, real_points1, purcell_open_end6 = meander.meander_creation2(name, begin_point, begin_orientation,
                                    end_point, end_orientation, meander_length,
                                    left_limit, right_limit, radius_limit, w,
                                    s, g, eqd.sample.layer_configuration)
eqd.sample.add(purcell_open_end6)
eqd.sample.generate_bridge_over_cpw(name='purcell_open_end6_airbridges', o=purcell_open_end6, geometry=airbridge,
                                 min_spacing=resonator_min_bridge_spacing, bridge_part_decimation=2)

connection = eqd.sample.connect_cpw(purcell_open_end6,
                           TL_purcell_couplers[1][0],
                           'port1',
                           'down',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=100)
print(connection[0].length)

# Add open end for purcell 
eqd.sample.open_end(purcell_open_end6, 'port2', 'purcell6-open-end')

A_max, B_max 954.25 952.25
2099.918169536469
A, K, B =  954.2499999999999 1041.9181695364691 952.2499999999999
A_max, B_max 1006.625 1004.625
1578.9590847682346
A, K, B =  789.4795423841173 520.9590847682346 789.4795423841173
points [(8411.0, 2303.0), (8411.0, 1461.1454576158826), (8515.75, 1461.1454576158826), (8515.75, 2086.854542384117), (8620.5, 2086.854542384117), (8620.5, 1461.1454576158826), (8725.25, 1461.1454576158826), (8725.25, 2086.854542384117), (8830.0, 2086.854542384117), (8830, 1245)] real points [(8411.0, 2303.0), (8411.0, 1513.5204576158826), (8515.75, 1513.5204576158826), (8515.75, 2034.4795423841172), (8620.5, 2034.4795423841172), (8620.5, 1513.5204576158826), (8725.25, 1513.5204576158826), (8725.25, 2034.4795423841172), (8830.0, 2034.4795423841174), (8830, 1245)] -1.5707963267948966 1.5707963267948966
10.0


OpenEnd purcell6-open-end

In [101]:
# Add coupler purcell + resonator with fanout and grounding
pur_res_coupler6_position = [
    (TL_purcell_couplers[1][1].terminals['up'].position[0] +
     purcell_resonator_coupler_length,
     TL_purcell_couplers[1][1].terminals['up'].position[1] - 225),
    (TL_purcell_couplers[1][1].terminals['up'].position[0],
     TL_purcell_couplers[1][1].terminals['up'].position[1] - 225)
]
pur_res_coupler6 = elements.CPWCoupler('purcell_resonator_coupler6',
                                         pur_res_coupler6_position,
                                         pur_res_coupler_w,
                                         pur_res_coupler_s,
                                         eqd.resonator_ground,
                                         eqd.sample.layer_configuration,
                                         r=0)
eqd.sample.add(pur_res_coupler6)

pur_res_coupler6_fanout = eqd.sample.fanout(o=pur_res_coupler6,
                                            port='port1',
                                            name='res-purcell-coupler-fanout',
                                            grouping=[0, 2])
pur_res_coupler6_ground = eqd.sample.ground(
    o=pur_res_coupler6,
    port='port2',
    name='res-purcell-coupler-grounding',
    grounding_width=res_pur_coupler_grounding,
    grounding_between=[(0, 4)])

pur_res_coupler6_fanout_ground = eqd.sample.ground(
    o=pur_res_coupler6_fanout,
    port='center',
    name='res-purcell-coupler-fanout-grounding',
    grounding_width=res_pur_coupler_grounding,
    grounding_between=[(0, 1)])

In [102]:
# Add closed end purcell
name = 'purcell_closed_end6'
begin_point = (pur_res_coupler6_fanout_ground.terminals['narrow'].position[0]+10,
               pur_res_coupler6_fanout_ground.terminals['narrow'].position[1])
begin_orientation = 0
end_point = (TL_purcell_couplers[1][1].terminals['up'].position[0]+120,
            TL_purcell_couplers[1][1].terminals['up'].position[1]-100)
end_orientation =   0
meander_length = 500
left_limit = 100
right_limit = 500
radius_limit = (40,90)
w = eqd.resonator_core
s = eqd.resonator_gap
g = eqd.resonator_ground

result1, points1, real_points1, purcell_closed_end6 = meander.meander_creation2(name, begin_point, begin_orientation,
                                    end_point, end_orientation, meander_length,
                                    left_limit, right_limit, radius_limit, w,
                                    s, g, eqd.sample.layer_configuration)
eqd.sample.add(purcell_closed_end6)
eqd.sample.generate_bridge_over_cpw(name='purcell_closed_end6_airbridges', o=purcell_closed_end6, geometry=airbridge,
                                 min_spacing=resonator_min_bridge_spacing, bridge_part_decimation=2)

connection = eqd.sample.connect_cpw(purcell_closed_end6,
                           pur_res_coupler6_fanout_ground,
                           'port1',
                           'narrow',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=100)
print(connection[0].length)

A_max, B_max 437.0 559.0
A, K, B =  168.31859100666537 0 168.31859100666537
points [(8031.0, 2109.0), (8251.318591006666, 2109.0), (8251.318591006666, 2213.0), (7909.0, 2213.0)] real points [(8031.0, 2109.0), (8199.318591006666, 2109.0), (8077.318591006665, 2213.0), (7909.0, 2213.0)] 0 0
10.0


In [103]:
# Add resonator
name = 'resonator6'
begin_point = (pur_res_coupler6_fanout.terminals['down'].position[0],
               pur_res_coupler6_fanout.terminals['down'].position[1]-10)
begin_orientation = - np.pi  / 2
end_point = (eqd.coaxmon6.terminals['coupler1'].position[0]+60,
            eqd.coaxmon6.terminals['coupler1'].position[1]+100)
end_orientation =   np.pi / 2
meander_length = 3950
left_limit = 810
right_limit = 10
radius_limit = (40,90)
w = eqd.resonator_core
s = eqd.resonator_gap
g = eqd.resonator_ground

result1, points1, real_points1, resonator6 = meander.meander_creation2(name, begin_point, begin_orientation,
                                    end_point, end_orientation, meander_length,
                                    left_limit, right_limit, radius_limit, w,
                                    s, g, eqd.sample.layer_configuration)
eqd.sample.add(resonator6)
eqd.sample.generate_bridge_over_cpw(name='resonator6_airbridges', o=resonator6, geometry=airbridge,
                                 min_spacing=resonator_min_bridge_spacing, bridge_part_decimation=2)

connection = eqd.sample.connect_cpw(resonator6,
                           pur_res_coupler6_fanout,
                           'port1',
                           'down',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=100)
print(connection[0].length)

A_max, B_max 717.2592385882797 268.0686305610109
993.8108684296518
A, K, B =  717.2592385882796 643.0014764569206 268.0686305610108
A_max, B_max 744.5061590588531 295.3155510315843
779.4770429440116
A, K, B =  538.8877403490286 428.6676509712804 240.5893025949829
A_max, B_max 758.1296192941398 308.93901126687103
672.3101302011914
A, K, B =  390.3008596476846 321.5007382284603 282.0092705535069
points [(8000.0, 2036.0), (8000.0, 1442.6184187098245), (7891.012318117706, 1442.6184187098245), (7891.012318117706, 1980.2737515633985), (7782.024636235413, 1980.2737515633985), (7782.024636235413, 1442.6184187098245), (7673.036954353119, 1442.6184187098245), (7673.036954353119, 1980.2737515633985), (7564.049272470825, 1980.2737515633985), (7564.049272470825, 1442.6184187098245), (7455.0615905885315, 1442.6184187098245), (7455.0615905885315, 1980.2737515633985), (7346.0739087062375, 1980.2737515633985), (7346.0739087062375, 1685.1906080272688)] real points [(8000.0, 2036.0), (8000.0, 1497.112259

In [104]:
connection = eqd.sample.connect_cpw(resonator6,
                           eqd.coaxmon6,
                           'port2',
                           'coupler1',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=100)
print(connection[0].length)

122.14889754564834


In [105]:
connection = eqd.sample.connect_cpw(TL_purcell_couplers[1][1],
                           purcell_closed_end6,
                           'up',
                           'port2',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=90)
print(connection[0].length)

179.60931522485987


In [106]:
eqd.sample.draw_design()
# eqd.sample.watch()

In [107]:
# eqd.sample.watch()

### Purcell and resonator for qubit 2

In [144]:
# Add purcell open end
eqd.sample.open_end(TL_purcell_couplers[6][0], 'up', 'purcell2-open-end')

OpenEnd purcell2-open-end

In [145]:
# Add coupler purcell + resonator with fanout and grounding
pur_res_coupler2_position = [(eqd.coaxmon5.center[0]-220 - purcell_resonator_coupler_length,
                              eqd.coaxmon5.center[1]+440),
                             (eqd.coaxmon5.center[0]-220,
                              eqd.coaxmon5.center[1]+440)]
pur_res_coupler2 = elements.CPWCoupler('purcell_resonator_coupler2',
                                         pur_res_coupler2_position,
                                         pur_res_coupler_w,
                                         pur_res_coupler_s,
                                         eqd.resonator_ground,
                                         eqd.sample.layer_configuration,
                                         r=0)
eqd.sample.add(pur_res_coupler2)

pur_res_coupler2_fanout = eqd.sample.fanout(o=pur_res_coupler2,
                                            port='port1',
                                            name='res-purcell-coupler-fanout',
                                            grouping=[1, 3])
pur_res_coupler2_ground = eqd.sample.ground(
    o=pur_res_coupler2,
    port='port2',
    name='res-purcell-coupler-grounding',
    grounding_width=res_pur_coupler_grounding,
    grounding_between=[(0, 4)])

pur_res_coupler2_fanout_ground = eqd.sample.ground(
    o=pur_res_coupler2_fanout,
    port='center',
    name='res-purcell-coupler-fanout-grounding',
    grounding_width=res_pur_coupler_grounding,
    grounding_between=[(2, 3)])

In [146]:
# Add resonator for qubit 2
name = 'resonator2'
begin_point = (pur_res_coupler2_fanout.terminals['up'].position[0],
               pur_res_coupler2_fanout.terminals['up'].position[1]-5)
begin_orientation = - np.pi / 2

end_point = (eqd.coaxmon2.terminals['coupler1'].position[0]+100,
             eqd.coaxmon2.terminals['coupler1'].position[1]+100)
end_orientation =   np.pi / 2
meander_length = 3100
left_limit = 330
right_limit = 0
radius_limit = (45,80)
w = eqd.resonator_core
s = eqd.resonator_gap
g = eqd.resonator_ground
result1, points1, real_points1, resonator2 = meander.meander_creation2(name, begin_point, begin_orientation,
                                    end_point, end_orientation, meander_length,
                                    left_limit, right_limit, radius_limit, w,
                                    s, g, eqd.sample.layer_configuration)

eqd.sample.add(resonator2)
eqd.sample.generate_bridge_over_cpw(name='resonator2_airbridges', o=resonator2, geometry=airbridge,
                                 min_spacing=resonator_min_bridge_spacing, bridge_part_decimation=2)

connection = eqd.sample.connect_cpw(resonator2,
                           pur_res_coupler2_fanout,
                           'port1',
                           'up',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=80)
print(connection[0].length)

A_max, B_max 245.42282572551977 73.23221769825098
417.02498313800635
A, K, B =  245.42282572551974 259.21559116527516 73.23221769825096
A_max, B_max 263.81711929413984 91.62651126687103
352.2210853466876
A, K, B =  262.1756917389896 194.41169337395638 90.04539360769796
points [(5069.0, 1843.0), (5069.0, 1525.6414275551501), (4958.63423858828, 1525.6414275551501), (4958.63423858828, 1830.4188823408267), (4848.26847717656, 1830.4188823408267), (4848.26847717656, 1525.6414275551501), (4737.902715764839, 1525.6414275551501), (4737.902715764839, 1830.4188823408267), (4627.536954353119, 1830.4188823408267), (4627.536954353119, 1525.6414275551501), (4517.171192941399, 1525.6414275551501), (4517.171192941399, 1830.4188823408267), (4406.805431529678, 1830.4188823408267), (4406.805431529678, 1525.6414275551501), (4296.439670117958, 1525.6414275551501), (4296.439670117958, 1830.4188823408267), (4186.0739087062375, 1830.4188823408267), (4186.0739087062375, 1685.1906080272688)] real points [(5069.0

In [147]:
# Add purcell closed end
name = 'purcell_closed_end2'
begin_point = (pur_res_coupler2_fanout_ground.terminals['narrow'].position[0]-1035,
               pur_res_coupler2_fanout_ground.terminals['narrow'].position[1]+ 50)
end_point = (TL_purcell_couplers[6][1].terminals['down'].position[0],
             TL_purcell_couplers[6][1].terminals['down'].position[1]-10)
begin_orientation = np.pi / 2
end_orientation = - np.pi / 2
meander_length = 2650
left_limit = 10
right_limit = 295
radius_limit = (40, 90)
w = eqd.resonator_core
s = eqd.resonator_gap
g = eqd.resonator_ground

result1, points1, real_points1, purcell_closed_end_2 = meander.meander_creation2(
    name, begin_point, begin_orientation, end_point, end_orientation,
    meander_length, left_limit, right_limit, radius_limit, w, s, g,
    eqd.sample.layer_configuration)
eqd.sample.add(purcell_closed_end_2)
eqd.sample.generate_bridge_over_cpw(name='purcell_closed_end_2_airbridges', o=purcell_closed_end_2, geometry=airbridge,
                                 min_spacing=resonator_min_bridge_spacing, bridge_part_decimation=2)


connection = eqd.sample.connect_cpw(purcell_closed_end_2,
                           TL_purcell_couplers[6][1],
                           'port2',
                           'down',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=50)
print(connection[0].length)

A_max, B_max 186.75 211.75
589.4801144383925
A, K, B =  186.74999999999997 279.4801144383926 211.74999999999997
A_max, B_max 219.16666666666669 244.16666666666669
496.3200762922618
A, K, B =  219.16666666666592 186.32007629226177 244.16666666666603
A_max, B_max 235.375 260.375
449.7400572191963
A, K, B =  224.87002860959814 139.7400572191963 224.87002860959814
points [(4013.0, 1961.0), (4013.0, 2234.495028609598), (4110.25, 2234.495028609598), (4110.25, 1997.5049713904018), (4207.5, 1997.5049713904018), (4207.5, 2234.495028609598), (4304.75, 2234.495028609598), (4304.75, 1997.5049713904018), (4402.0, 1997.5049713904018), (4402.0, 2234.495028609598), (4499.25, 2234.495028609598), (4499.25, 1997.5049713904018), (4596.5, 1997.5049713904018), (4596.5, 2234.495028609598), (4693.75, 2234.495028609598), (4693.75, 1997.5049713904018), (4791.0, 1997.5049713904018), (4791.0, 2271.0)] real points [(4013.0, 1961.0), (4013.0, 2185.870028609598), (4110.25, 2185.870028609598), (4110.25, 2046.12997139

In [148]:
purcell_closed_end_2_line = elements.CPW('purcell_closed_end_2_line',
                                        [(pur_res_coupler2_fanout_ground.terminals['narrow'].position[0],
                                          pur_res_coupler2_fanout_ground.terminals['narrow'].position[1]),
                                         (pur_res_coupler2_fanout_ground.terminals['narrow'].position[0]-955,
                                          pur_res_coupler2_fanout_ground.terminals['narrow'].position[1])],
                                         eqd.resonator_core,
                                         eqd.resonator_gap,
                                         eqd.resonator_ground,
                                         eqd.sample.layer_configuration,
                                         r=0)

eqd.sample.add(purcell_closed_end_2_line)
eqd.sample.generate_bridge_over_cpw(name='purcell_closed_end_2_line_airbridges', o=purcell_closed_end_2_line, geometry=airbridge,
                                 min_spacing=resonator_min_bridge_spacing, bridge_part_decimation=2)

eqd.sample.connect(purcell_closed_end_2_line, 'port1', pur_res_coupler2_fanout_ground, 'narrow')
purcell_closed_end_2_line.length

955.0

In [113]:
connection = eqd.sample.connect_cpw(purcell_closed_end_2,
                           purcell_closed_end_2_line,
                           'port1',
                           'port2',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=50)
print(connection[0].length)

connection = eqd.sample.connect_cpw(eqd.coaxmon2,
                           resonator2,
                           'coupler1',
                           'port2',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=50)
print(connection[0].length)

108.59679776502796
150.8106560562621


In [114]:
eqd.sample.draw_design()
# eqd.sample.watch()

In [115]:
# eqd.sample.write_to_gds()

### Purcell and resonator for qubit 8

In [116]:
# Add purcell open end
eqd.sample.open_end(TL_purcell_couplers[7][0], 'up', 'purcell2-open-end')

# Add coupler purcell + resonator with fanout and grounding
pur_res_coupler8_position = [(eqd.coaxmon3.center[0]+220 + purcell_resonator_coupler_length,
                              eqd.coaxmon3.center[1]-440),
                             (eqd.coaxmon3.center[0]+220,
                              eqd.coaxmon3.center[1]-440)]
pur_res_coupler8 = elements.CPWCoupler('purcell_resonator_coupler8',
                                         pur_res_coupler8_position,
                                         pur_res_coupler_w,
                                         pur_res_coupler_s,
                                         eqd.resonator_ground,
                                         eqd.sample.layer_configuration,
                                         r=0)
eqd.sample.add(pur_res_coupler8)

pur_res_coupler8_fanout = eqd.sample.fanout(o=pur_res_coupler8,
                                            port='port1',
                                            name='res-purcell-coupler-fanout',
                                            grouping=[1, 3])
pur_res_coupler8_ground = eqd.sample.ground(
    o=pur_res_coupler8,
    port='port2',
    name='res-purcell-coupler-grounding',
    grounding_width=res_pur_coupler_grounding,
    grounding_between=[(0, 4)])

pur_res_coupler8_fanout_ground = eqd.sample.ground(
    o=pur_res_coupler8_fanout,
    port='center',
    name='res-purcell-coupler-fanout-grounding',
    grounding_width=res_pur_coupler_grounding,
    grounding_between=[(2, 3)])

In [117]:
# Add resonator for qubit 8
name = 'resonator8'
begin_point = (pur_res_coupler8_fanout.terminals['up'].position[0],
               pur_res_coupler8_fanout.terminals['up'].position[1]+5)
begin_orientation = np.pi / 2

end_point = (eqd.coaxmon8.terminals['coupler1'].position[0]-100,
             eqd.coaxmon8.terminals['coupler1'].position[1]-100)
end_orientation =   - np.pi / 2
meander_length = 3125
left_limit = 0
right_limit = 330
radius_limit = (45,80)
w = eqd.resonator_core
s = eqd.resonator_gap
g = eqd.resonator_ground
result1, points1, real_points1, resonator8 = meander.meander_creation2(name, begin_point, begin_orientation,
                                    end_point, end_orientation, meander_length,
                                    left_limit, right_limit, radius_limit, w,
                                    s, g, eqd.sample.layer_configuration)

eqd.sample.add(resonator8)
eqd.sample.generate_bridge_over_cpw(name='resonator8_airbridges', o=resonator8, geometry=airbridge,
                                 min_spacing=resonator_min_bridge_spacing, bridge_part_decimation=2)

connection = eqd.sample.connect_cpw(resonator8,
                           pur_res_coupler8_fanout,
                           'port1',
                           'up',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=80)
print(connection[0].length)

A_max, B_max 245.42282572551977 73.23221769825098
421.19164980467303
A, K, B =  245.42282572551974 263.38225783194184 73.23221769825096
A_max, B_max 263.81711929413984 91.62651126687103
355.3460853466876
A, K, B =  263.76831905709184 197.53669337395638 91.57776628875402
points [(4331.0, 2857.0), (4331.0, 3175.951199762952), (4441.36576141172, 3175.951199762952), (4441.36576141172, 2868.0487449772754), (4551.73152282344, 2868.0487449772754), (4551.73152282344, 3175.951199762952), (4662.097284235161, 3175.951199762952), (4662.097284235161, 2868.0487449772754), (4772.463045646881, 2868.0487449772754), (4772.463045646881, 3175.951199762952), (4882.828807058601, 3175.951199762952), (4882.828807058601, 2868.0487449772754), (4993.194568470322, 2868.0487449772754), (4993.194568470322, 3175.951199762952), (5103.560329882042, 3175.951199762952), (5103.560329882042, 2868.0487449772754), (5213.9260912937625, 2868.0487449772754), (5213.9260912937625, 3014.809391972731)] real points [(4331.0, 2857.0

In [118]:
# Add purcell closed end
name = 'purcell_closed_end8'
begin_point = (pur_res_coupler8_fanout_ground.terminals['narrow'].position[0]+1035,
               pur_res_coupler8_fanout_ground.terminals['narrow'].position[1]- 55)
end_point = (TL_purcell_couplers[7][1].terminals['down'].position[0],
             TL_purcell_couplers[7][1].terminals['down'].position[1]+10)
begin_orientation = - np.pi / 2
end_orientation =  np.pi / 2
meander_length = 2900
left_limit = 300
right_limit = 0
radius_limit = (40, 90)
w = eqd.resonator_core
s = eqd.resonator_gap
g = eqd.resonator_ground

result1, points1, real_points1, purcell_closed_end_8 = meander.meander_creation2(
    name, begin_point, begin_orientation, end_point, end_orientation,
    meander_length, left_limit, right_limit, radius_limit, w, s, g,
    eqd.sample.layer_configuration)
eqd.sample.add(purcell_closed_end_8)
eqd.sample.generate_bridge_over_cpw(name='purcell_closed_end_8_airbridges', o=purcell_closed_end_8, geometry=airbridge,
                                 min_spacing=resonator_min_bridge_spacing, bridge_part_decimation=2)


connection = eqd.sample.connect_cpw(purcell_closed_end_8,
                           TL_purcell_couplers[7][1],
                           'port2',
                           'down',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=50)
print(connection[0].length)

A_max, B_max 191.75 196.75
648.2301144383925
A, K, B =  191.74999999999997 343.2301144383926 196.74999999999997
A_max, B_max 224.16666666666669 229.16666666666669
533.8200762922618
A, K, B =  224.16666666666666 228.82007629226177 229.16666666666666
A_max, B_max 240.375 245.375
476.6150572191963
A, K, B =  238.30752860959814 171.6150572191963 238.30752860959814
points [(5387.0, 2734.0), (5387.0, 2447.067471390402), (5289.75, 2447.067471390402), (5289.75, 2715.932528609598), (5192.5, 2715.932528609598), (5192.5, 2447.067471390402), (5095.25, 2447.067471390402), (5095.25, 2715.932528609598), (4998.0, 2715.932528609598), (4998.0, 2447.067471390402), (4900.75, 2447.067471390402), (4900.75, 2715.932528609598), (4803.5, 2715.932528609598), (4803.5, 2447.067471390402), (4706.25, 2447.067471390402), (4706.25, 2715.932528609598), (4609.0, 2715.932528609598), (4609.0, 2429.0)] real points [(5387.0, 2734.0), (5387.0, 2495.692471390402), (5289.75, 2495.692471390402), (5289.75, 2667.307528609598), (

In [119]:
purcell_closed_end_8_line = elements.CPW('purcell_closed_end_8_line',
                                        [(pur_res_coupler8_fanout_ground.terminals['narrow'].position[0],
                                          pur_res_coupler8_fanout_ground.terminals['narrow'].position[1]),
                                         (pur_res_coupler8_fanout_ground.terminals['narrow'].position[0]+955,
                                          pur_res_coupler8_fanout_ground.terminals['narrow'].position[1])],
                                         eqd.resonator_core,
                                         eqd.resonator_gap,
                                         eqd.resonator_ground,
                                         eqd.sample.layer_configuration,
                                         r=0)

eqd.sample.add(purcell_closed_end_8_line)
eqd.sample.generate_bridge_over_cpw(name='purcell_closed_end_8_line_airbridges', o=purcell_closed_end_8_line, geometry=airbridge,
                                 min_spacing=resonator_min_bridge_spacing, bridge_part_decimation=2)

eqd.sample.connect(purcell_closed_end_8_line, 'port1', pur_res_coupler8_fanout_ground, 'narrow')

In [120]:
connection = eqd.sample.connect_cpw(purcell_closed_end_8,
                           purcell_closed_end_8_line,
                           'port1',
                           'port2',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=50)
print(connection[0].length)

connection = eqd.sample.connect_cpw(eqd.coaxmon8,
                           resonator8,
                           'coupler1',
                           'port2',
                           'TL', [],
                           airbridge=airbridge,
                           min_spacing=coupler_bridge_spacing,
                           r=50)
print(connection[0].length)

112.994316255421
150.8106560562647


In [121]:
eqd.sample.draw_design()
# eqd.sample.watch()

In [122]:
eqd.sample.watch()

# Add grid (vortex trap)

In [114]:
grid_ground = elements.GridGround(eqd.sample.chip_geometry,
                                  eqd.sample.layer_configuration,
                                  eqd.sample.objects,
                                  width=5,
                                  period=25)
eqd.sample.add(grid_ground)

In [ ]:
eqd.sample.draw_design()
eqd.sample.watch()

In [120]:
eqd.sample.write_to_gds()

Gds file has been writen here:  /home/alena/QCreator/designs/8qubit_desigh/SQC8-script.gds


# tlsim simulations

## Calculate qubits capacitances  using fastcap

In [78]:
eqd.sample.qubits

In [79]:
eqd.sample.draw_cap()
for i, qubit in enumerate(eqd.sample.qubits):
    caps = eqd.sample.calculate_qubit_capacitance(
        cell=eqd.sample.qubit_cap_cells[i],
        qubit=eqd.sample.qubits[i],
        mesh_volume=10)

Gds file has been writen here:  /home/alena/QCreator/designs/8qubit_desigh/SQC8-script.gds
For 0 conductor total length is:  10314
For 1 conductor total length is:  4892
For 2 conductor total length is:  1605
For 3 conductor total length is:  943
For 4 conductor total length is:  931
Sum : 18685
Data has been written into the file: /home/alena/QCreator/designs/8qubit_desigh/mesh_4k_data
Time for fastcap job is:  2.4662957191467285
Capacitance results have been writen here:  /home/alena/QCreator/designs/8qubit_desigh/mesh_4k_results
Capacitance value in:  femtofarads
[<map object at 0x7fd450331d60>, <map object at 0x7fd450331f10>, <map object at 0x7fd450333730>, <map object at 0x7fd450333e20>, <map object at 0x7fd450333310>]
       0     1     2     3     4
0  151.8 -33.5 -16.1 -11.8 -12.5
1  -33.5  66.3  -7.8  -5.2  -5.3
2  -16.1  -7.8  40.8  -5.1  -4.0
3  -11.8  -5.2  -5.1  27.8  -0.3
4  -12.5  -5.3  -4.0  -0.3  27.7
Gds file has been writen here:  /home/alena/QCreator/designs/8qubit_

## Calculate modes 

In [80]:
# Create transmission line model
sys, connections, elements_ = eqd.sample.get_tls(cutoff=1e11)

In [81]:
omegas, kappas, modes = sys.get_modes()

In [83]:
omegas / (2 * np.pi)

array([4.90417390e+09, 4.91239989e+09, 4.96487094e+09, 4.96664548e+09,
       4.96671045e+09, 4.96988398e+09, 4.96991001e+09, 4.97107008e+09,
       5.51357254e+09, 6.92115366e+09, 7.03331420e+09, 7.97907264e+09,
       8.61890175e+09, 1.29530236e+10, 1.30542058e+10, 1.30558646e+10,
       1.31107551e+10, 1.31122192e+10, 1.41954137e+10, 1.41966978e+10,
       1.42262127e+10, 1.42264404e+10, 1.46040085e+10, 1.46075518e+10,
       1.60727708e+10, 1.61297156e+10, 1.78405021e+10, 1.78409297e+10,
       1.78825476e+10, 1.78834247e+10, 1.81536443e+10, 1.81599984e+10,
       1.82742875e+10, 1.82805754e+10, 1.85223115e+10, 1.85297897e+10,
       1.93464866e+10, 1.97075850e+10, 2.18118238e+10, 2.38259654e+10,
       2.56962567e+10, 2.56962568e+10, 2.77127418e+10, 2.82019314e+10,
       2.84164319e+10, 2.85674761e+10, 2.89232586e+10, 2.90058239e+10,
       2.97412146e+10, 2.98434840e+10, 3.08384703e+10, 3.09816631e+10,
       3.24252892e+10, 3.31663896e+10, 3.35266801e+10, 3.46310564e+10,
      

In [88]:
len(omegas)

600

In [84]:
import pandas as pd

In [93]:
frequencies = pd.DataFrame(omegas / (2 * np.pi) / 1e9, columns = ['frequency, GHz'])
frequencies.to_csv('8_qubit_frequencies.csv')

In [100]:
relaxation_rates = pd.DataFrame(kappas / (2 * np.pi), columns = ['relaxation_rates, Hz'])
relaxation_rates.to_csv('8_qubit_relaxation_rates.csv')

In [106]:
modes_vectors = pd.DataFrame(modes)
modes_vectors.to_csv('modes_vectors.csv')
modes_vectors

,0,1,2,3,4,5,6,7,8,9,...,4464,4465,4466,4467,4468,4469,4470,4471,4472,4473
0,0.0+0.0j,-1.080512e-07+4.942777e-08j,2.652449e-07+1.948372e-07j,3.692369e-06-2.083138e-06j,3.627874e-13-1.886257e-13j,1.693110e-12+2.149270e-12j,-1.634429e-12+7.406914e-13j,-2.068904e-05+6.834684e-06j,4.104200e-06-1.294154e-06j,-1.240423e-07+2.530572e-08j,...,-2.199274e-09+5.104503e-07j,1.076695e-11-2.485793e-09j,4.666928e-06+2.019370e-08j,4.328456e-07+1.864862e-09j,-2.186656e-09+5.075356e-07j,6.646762e-12-1.536114e-09j,-9.650666e-06+8.850895e-10j,2.265180e-07-1.174822e-11j,5.954583e-12+1.146874e-07j,6.748624e-13+7.356489e-09j
1,0.0+0.0j,8.760857e-08-4.015343e-08j,-1.306323e-05-9.555838e-06j,-5.956296e-08+3.370621e-08j,-4.062153e-10+2.304528e-10j,-9.047569e-10-1.248239e-09j,9.366055e-12-4.300022e-12j,-1.015440e-08+3.360970e-09j,-1.020468e-07+3.223552e-08j,1.498559e-07-3.062445e-08j,...,1.098032e-07+4.920720e-06j,-5.393415e-10-5.536432e-08j,9.253939e-05-1.009872e-06j,4.150989e-06-9.326139e-08j,1.091711e-07+4.859116e-06j,-3.329558e-10-3.051034e-08j,2.326715e-02-2.060482e-08j,-5.399473e-04+4.017068e-11j,-2.167467e-11-2.729207e-04j,-1.564397e-11-1.776578e-05j
2,0.0+0.0j,1.119291e-09-5.192247e-10j,-1.556832e-08-1.107843e-08j,-8.532762e-10+4.885910e-10j,1.233898e-05-7.089416e-06j,1.474718e-06+2.005167e-06j,-1.427986e-06+6.623891e-07j,2.819558e-10-9.437465e-11j,3.440673e-08-1.099199e-08j,-4.850261e-08+1.002033e-08j,...,1.434858e-10+5.655947e-09j,-7.202724e-13-6.322261e-11j,1.048245e-07-1.334272e-09j,4.822498e-09-1.231561e-10j,1.426417e-10+5.585519e-09j,-4.446394e-13-3.492835e-11j,2.559831e-05-2.068316e-10j,-5.854448e-07+4.173447e-12j,-2.080117e-12-2.927903e-07j,-1.592671e-13-1.975454e-08j
3,0.0+0.0j,2.569923e-08-1.192550e-08j,2.155233e-07+1.532256e-07j,-3.879036e-08+2.221826e-08j,-1.000358e-06+5.750462e-07j,-2.876580e-08-3.908890e-08j,3.919999e-08-1.819526e-08j,1.130429e-07-3.785325e-08j,1.389696e-05-4.441337e-06j,-1.957485e-05+4.045472e-06j,...,-1.991551e-09-7.990668e-08j,9.999556e-12+1.302652e-10j,-3.368201e-07+1.851863e-08j,-6.885907e-08+1.709986e-09j,-1.979832e-09-7.972553e-08j,6.173015e-12+1.122709e-10j,1.990210e-04-7.936212e-10j,-4.549930e-06+2.629919e-11j,-1.315662e-11-2.274680e-06j,-6.126711e-13-1.536419e-07j
4,0.0+0.0j,2.118964e-09-9.833534e-10j,1.598366e-07+1.136254e-07j,-9.353459e-09+5.357592e-09j,2.564388e-06-1.474035e-06j,1.335534e-07+1.814973e-07j,-3.588061e-07+1.665076e-07j,4.353787e-08-1.458069e-08j,5.351776e-06-1.710582e-06j,-7.538165e-06+1.558150e-06j,...,-1.478537e-09-5.838082e-08j,7.422576e-12+3.494650e-10j,-6.273132e-07+1.374640e-08j,-5.007564e-08+1.269517e-09j,-1.469838e-09-5.797718e-08j,4.582362e-12+2.091186e-10j,-4.134538e-05-1.598155e-09j,9.448957e-07+4.255487e-11j,-2.127997e-11+4.723825e-07j,-1.234309e-12+3.191864e-08j
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.0+0.0j,-6.502618e-28-4.023540e-29j,4.940982e-19+3.933770e-20j,1.698004e-21-2.086334e-20j,3.467729e-19-1.462631e-19j,-3.595863e-19-9.976123e-21j,-5.189869e-20+9.620604e-20j,4.571299e-30+2.531905e-31j,5.233327e-19+1.762093e-19j,2.102453e-17+1.143617e-16j,...,-1.735236e-18-2.007752e-10j,-1.050308e-15+5.701997e-07j,-1.023188e-03+6.907902e-17j,-5.283337e-03-7.276812e-14j,2.230363e-17-1.350409e-06j,4.556334e-17+1.544986e-03j,2.455487e-15-8.597726e-16j,2.835261e-15-1.612738e-15j,1.268358e-19+2.352628e-19j,-3.006560e-15-8.524832e-15j
596,0.0+0.0j,2.514558e-29+1.086590e-30j,9.283759e-18+2.768722e-20j,-4.306577e-21+1.415036e-20j,-1.321275e-18+1.498606e-18j,-1.954886e-19-3.001474e-20j,1.711970e-19-1.969234e-19j,1.939108e-32-1.584163e-31j,1.968668e-18-3.770785e-18j,8.336738e-17-1.495935e-15j,...,1.244267e-19+4.178180e-20j,-4.495632e-16-6.304813e-17j,3.989514e-15+6.356878e-15j,2.245924e-14+3.620716e-14j,-9.443463e-18+4.568203e-18j,1.226268e-14-1.714077e-15j,5.204019e-16+1.514102e-16j,1.374962e-15-1.023247e-15j,4.181374e-18+1.068069e-19j,6.808671e-16-2.339220e-15j
597,0.0+0.0j,-2.508192e-28-1.065929e-29j,-4.896669e-21-7.575062e-21j,-2.9496